[**Connect 4**](https://www.kaggle.com/competitions/connect-4)

[**Connect X**](https://www.kaggle.com/competitions/connectx/leaderboard)


[debugger](https://www.kaggle.com/vyacheslavbolotin/debugger-c4)

[round_robin](https://www.kaggle.com/code/dott1718/kdb-workshop)

[agent_Connect X - treebased](https://www.kaggle.com/code/dott1718/kdb-workshop) - [*dott*](https://www.kaggle.com/dott1718)

[agent_Connect_X - MCTS](https://www.kaggle.com/code/matant/monte-carlo-tree-search-connectx) - [*Matan Tsipory*](https://www.kaggle.com/matant)

[agent_Connect_X - MCTS w/ Adaptive Playouts](https://www.kaggle.com/code/glazed/monte-carlo-w-adaptive-playouts) - [*glazed*](https://www.kaggle.com/glazed)

[agent_Connect_X - Monty Carlo Tree Search](https://www.kaggle.com/code/matant/monte-carlo-tree-search-connectx) - [*James McGuigan*](https://www.kaggle.com/jamesmcguigan)

[agent_Connect_X - MCTS Bitboard + Bitsquares Heuristic](https://www.kaggle.com/code/jamesmcguigan/connectx-mcts-bitboard-bitsquares-heuristic) - [*James McGuigan*](https://www.kaggle.com/jamesmcguigan)

[agent_Connect_X - The Fast mini Max II](https://www.kaggle.com/code/martynovandrey/the-fast-mini-max-ii) - [The Fast mini Max IV](https://www.kaggle.com/code/martynovandrey/the-fast-mini-max-iv) - [*Martynov Andrey*](https://www.kaggle.com/martynovandrey)

In [1]:
from kaggle_environments import make, evaluate

No pygame installed, ignoring import


### [round_robin](https://www.kaggle.com/code/dott1718/kdb-workshop)

In [2]:
import random
import numpy as np
import pandas as pd
from joblib import Parallel, delayed


def create(): 
    return make("connectx", configuration={"inarow":4,"columns":7,"rows":6})
       
    
def run_game(agent1, agent2, shuffle=False):
    env = create()
    swap = shuffle and random.choice([True, False])
    if swap:
        env.run([agent2, agent1])
    else:
        env.run([agent1, agent2])

    if env.steps[-1][0].status == "TIMEOUT":
        rew1 = 0
        rew2 = 1
    elif env.steps[-1][1].status == "TIMEOUT":
        rew2 = 0
        rew1 = 1
    else:
        rew1 = (env.steps[-1][0].reward + 1.0) / 2
        rew2 = (env.steps[-1][1].reward + 1.0) / 2

    time1 = max(0, env.steps[-1][0].observation.remainingOverageTime)
    time2 = max(0, env.steps[-1][1].observation.remainingOverageTime)

    if swap:
        return rew2, rew1, time2, time1, len(env.steps)
    else:
        return rew1, rew2, time1, time2, len(env.steps)


def run_games(agent1, agent2, n_games=1, n_jobs=-1, shuffle=True):
    if n_jobs != 1:
        game_res = Parallel(
            n_jobs=-1, temp_folder="/tmp", max_nbytes=None, backend="loky"
        )(delayed(run_game)(agent1, agent2, shuffle) for _ in range(n_games))
    else:
        game_res = [run_game(agent1, agent2, shuffle) for _ in range(n_games)]

    return [np.mean([x[i] for x in game_res]) for i in range(5)]


def round_robin(agents, n_games=1, n_jobs=-1):
    res = {
        "name": [],
        "win_rate": [],
        "rem_time": [],
    }
    for i in range(len(agents)):
        for j in range(len(agents)):
            if j <= i:
                continue
            game_res = run_games(agents[i], agents[j], n_games, n_jobs)
            res["name"].append(agents[i] if type(agents[i]) == str else agents[i].__name__)
            res["win_rate"].append(game_res[0])
            res["rem_time"].append(game_res[2])
            res["name"].append(agents[j] if type(agents[j]) == str else agents[j].__name__)
            res["win_rate"].append(game_res[1])
            res["rem_time"].append(game_res[3])
    return (
        pd.DataFrame(res)
        .groupby("name")
        .mean()
        .reset_index()
        .sort_values(["win_rate", "rem_time"], ascending=False)
        .reset_index(drop=True)
    )

## [treebased](https://www.kaggle.com/code/dott1718/kdb-workshop) - [*dott*](https://www.kaggle.com/dott1718)

In [3]:
%%writefile agent_treebased_dott.py

def agent__treebased__dott__(observation, configuration, MAX_DEPTH=5, abp=True, shuffle_actions=True):
    import numpy as np

    MAX_SCORE = 9999
    ht = {}

    def score_block(d):
        if d[3] > 0:
            return 0
        if d[2] == configuration.inarow:
            return -MAX_SCORE
        if d[1] == configuration.inarow:
            return MAX_SCORE
        if (d[1] > 0) & (d[2] > 0):
            return 0
        if d[1] > 0:
            return d[1]
        else:
            return -d[2]

    def _score_seq(seq):
        sc = 0
        if len(seq) < configuration.inarow:
            return 0

        cnt = {0: 0, 1: 0, 2: 0, 3: 0}
        for i in range(configuration.inarow):
            cnt[seq[i]] += 1
        sc += score_block(cnt)
        for i in range(len(seq) - configuration.inarow):
            cnt[seq[i]] -= 1
            cnt[seq[i + configuration.inarow]] += 1
            sc += score_block(cnt)

        return sc

    def _score_delta(board, i, j):
        sc = 0
        for seq in [
            board[i, :],
            board[:, j],
            np.diagonal(board, j - i),
            np.diagonal(np.flip(board, axis=1), configuration.columns - j - 1 - i),
        ]:
            if len(seq) >= configuration.inarow:
                sc += ht[seq.data.tobytes()]

        return sc

    def _make_action(board, action, marker):
        new_board = board.copy()
        ind = np.where(board[:, action] == 0)[0][-1]
        new_board[ind, action] = marker
        return new_board, ind, action

    def _minimax(
        board,
        current_depth=1,
        base_score=0,
        last_action=None,
        alpha=-np.inf,
        beta=np.inf,
    ):

        if last_action is None:
            score = base_score
        else:
            score = base_score + _score_delta(board, last_action[0], last_action[1])

        if score <= -MAX_SCORE + 100:
            return score - MAX_DEPTH + current_depth, -1
        if score >= MAX_SCORE - 100:
            return score + MAX_DEPTH - current_depth, -1
        if current_depth == MAX_DEPTH:
            return score, -1

        marker = 1 if current_depth % 2 == 1 else 2

        actions = list(np.where(board[0, :] == 0)[0])
        if len(actions) == 0:
            return score, -1
        if shuffle_actions:
            np.random.shuffle(actions)

        is_max_turn = marker == 1
        best_score = -MAX_SCORE if is_max_turn else MAX_SCORE
        action_target = actions[0]

        for action in actions:
            new_board, a_i, a_j = _make_action(board, action, marker)
            child_score, child_action = _minimax(
                new_board,
                current_depth + 1,
                score - _score_delta(board, a_i, a_j),
                (a_i, a_j),
                alpha,
                beta,
            )

            if is_max_turn and best_score < child_score:
                best_score = child_score
                action_target = action
                if abp:
                    alpha = max(alpha, best_score)
                    if alpha >= beta:
                        break

            elif (not is_max_turn) and best_score > child_score:
                best_score = child_score
                action_target = action
                if abp:
                    beta = min(beta, best_score)
                    if beta <= alpha:
                        break

        return best_score, action_target

    # Pre-calculate hash-table of scores
    for l in np.arange(
        configuration.inarow, max(configuration.rows, configuration.columns) + 1
    ):
        for n in range(3**l):
            ar = np.zeros(l, dtype=int)
            nn = n
            for i in range(l):
                ar[i] = nn % 3
                nn = nn // 3
            ht[ar.data.tobytes()] = _score_seq(ar)

    # Import board and pre-process it
    board = np.array(observation.board, dtype=int).reshape(
        configuration.rows, configuration.columns
    )
    if observation.mark == 2:
        board = board * 2
        board[board == 4] = 1

    actions = (board == 0).sum(axis=0)

    # Run minmax
    res_score, res_action = _minimax(board)

    if res_action < 0:
        return np.where(actions > 0)[0][0]

    return int(res_action)

Writing agent_treebased_dott.py


In [4]:
%run agent_treebased_dott.py

## [MCTS](https://www.kaggle.com/code/matant/monte-carlo-tree-search-connectx) - [*Matan Tsipory*](https://www.kaggle.com/matant)

This is an agent that plays with UCT-MCTS. Code is highly commented to be readable.

In [5]:
%%writefile agent_MCTS_MatanTsipory.py

def agent_MCTS_MatanTsipory_(observation, configuration):
    """
    Connect X agent based on MCTS.
    """
    import random
    import math
    import time
    global current_state  # so tree can be recycled

    init_time = time.time()
    EMPTY = 0
    T_max = 1.0
    # T_max = configuration.timeout - 0.34  # time per move, left some overhead
    Cp_default = 1

    def play(board, column, mark, config):
        """ Plays a move. Taken from the Kaggle environment. """
        columns = config.columns
        rows = config.rows
        row = max([r for r in range(rows) if board[column + (r * columns)] == EMPTY])
        board[column + (row * columns)] = mark

    def is_win(board, column, mark, config):
        """ Checks for a win. Taken from the Kaggle environment. """
        columns = config.columns
        rows = config.rows
        inarow = config.inarow - 1
        row = min([r for r in range(rows) if board[column + (r * columns)] == mark])

        def count(offset_row, offset_column):
            for i in range(1, inarow + 1):
                r = row + offset_row * i
                c = column + offset_column * i
                if (
                        r < 0
                        or r >= rows
                        or c < 0
                        or c >= columns
                        or board[c + (r * columns)] != mark
                ):
                    return i - 1
            return inarow

        return (
                count(1, 0) >= inarow  # vertical.
                or (count(0, 1) + count(0, -1)) >= inarow  # horizontal.
                or (count(-1, -1) + count(1, 1)) >= inarow  # top left diagonal.
                or (count(-1, 1) + count(1, -1)) >= inarow  # top right diagonal.
        )

    def is_tie(board):
        """ Checks if a tie occured. """
        return not(any(mark == EMPTY for mark in board))

    def check_finish_and_score(board, column, mark, config):
        """ Returns a tuple where the first argument states whether game is finished and second argument returns score if game has finished. """
        if is_win(board, column, mark, config):
            return (True, 1)
        if is_tie(board):
            return (True, 0.5)
        else:
            return (False, None)

    def uct_score(node_total_score, node_total_visits, parent_total_visits, Cp=Cp_default):
        """ UCB1 calculation. """
        if node_total_visits == 0:
            return math.inf
        return node_total_score / node_total_visits + Cp * math.sqrt(
            2 * math.log(parent_total_visits) / node_total_visits)

    def opponent_mark(mark):
        """ The mark indicates which player is active - player 1 or player 2. """
        return 3 - mark

    def opponent_score(score):
        """ To backpropagate scores on the tree. """
        return 1 - score

    def random_action(board, config):
        """ Returns a random legal action (from the open columns). """
        return random.choice([c for c in range(config.columns) if board[c] == EMPTY])

    def default_policy_simulation(board, mark, config):
        """
        Run a random play simulation. Starting state is assumed to be a non-terminal state.
        Returns score of the game for the player with the given mark.
        """
        original_mark = mark
        board = board.copy()
        column = random_action(board, config)
        play(board, column, mark, config)
        is_finish, score = check_finish_and_score(board, column, mark, config)
        while not is_finish:
            mark = opponent_mark(mark)
            column = random_action(board, config)
            play(board, column, mark, config)
            is_finish, score = check_finish_and_score(board, column, mark, config)
        if mark == original_mark:
            return score
        return opponent_score(score)
    
    def find_action_taken_by_opponent(new_board, old_board, config):
        """ Given a new board state and a previous one, finds which move was taken. Used for recycling tree between moves. """
        for i, piece in enumerate(new_board):
            if piece != old_board[i]:
                return i % config.columns
        return -1  # shouldn't get here

    class State():
        """ 
        A class that represents nodes in the game tree.
        
        """
        def __init__(self, board, mark, config, parent=None, is_terminal=False, terminal_score=None, action_taken=None):
            self.board = board.copy()
            self.mark = mark
            self.config = config
            self.children = []
            self.parent = parent
            self.node_total_score = 0
            self.node_total_visits = 0
            self.available_moves = [c for c in range(config.columns) if board[c] == EMPTY]
            self.expandable_moves = self.available_moves.copy()
            self.is_terminal = is_terminal
            self.terminal_score = terminal_score
            self.action_taken = action_taken

        def is_expandable(self):
            """ Checks if the node has unexplored children. """
            return (not self.is_terminal) and (len(self.expandable_moves) > 0)

        def expand_and_simulate_child(self):
            """ Expands a random move from the legal unexplored moves, and runs a simulation of it 
            (Expansion + Simulation + Backpropagation stages in the MCTS algorithm description). """
            column = random.choice(self.expandable_moves)
            child_board = self.board.copy()
            play(child_board, column, self.mark, self.config)
            is_terminal, terminal_score = check_finish_and_score(child_board, column, self.mark, self.config)
            self.children.append(State(child_board, opponent_mark(self.mark),
                                       self.config, parent=self,
                                       is_terminal=is_terminal,
                                       terminal_score=terminal_score,
                                       action_taken=column
                                       ))
            simulation_score = self.children[-1].simulate()
            self.children[-1].backpropagate(simulation_score)
            self.expandable_moves.remove(column)

        def choose_strongest_child(self, Cp):
            """
            Chooses child that maximizes UCB1 score (Selection stage in the MCTS algorithm description).
            """
            children_scores = [uct_score(child.node_total_score,
                                         child.node_total_visits,
                                         self.node_total_visits,
                                         Cp) for child in self.children]
            max_score = max(children_scores)
            best_child_index = children_scores.index(max_score)
            return self.children[best_child_index]
            
        def choose_play_child(self):
            """ Choose child with maximum total score."""
            children_scores = [child.node_total_score for child in self.children]
            max_score = max(children_scores)
            best_child_index = children_scores.index(max_score)
            return self.children[best_child_index]

        def tree_single_run(self):
            """
            A single iteration of the 4 stages of the MCTS algorithm.
            """
            if self.is_terminal:
                self.backpropagate(self.terminal_score)
                return
            if self.is_expandable():
                self.expand_and_simulate_child()
                return
            self.choose_strongest_child(Cp_default).tree_single_run()

        def simulate(self):
            """
            Runs a simulation from the current state. 
            This method is used to simulate a game after move of current player, so if a terminal state was reached,
            the score would belong to the current player who made the move.
            But otherwise the score received from the simulation run is the opponent's score and thus needs to be flipped with the function opponent_score().            
            """
            if self.is_terminal:
                return self.terminal_score
            return opponent_score(default_policy_simulation(self.board, self.mark, self.config))

        def backpropagate(self, simulation_score):
            """
            Backpropagates score and visit count to parents.
            """
            self.node_total_score += simulation_score
            self.node_total_visits += 1
            if self.parent is not None:
                self.parent.backpropagate(opponent_score(simulation_score))
                
        def choose_child_via_action(self, action):
            """ Choose child given the action taken from the state. Used for recycling of tree. """
            for child in self.children:
                if child.action_taken == action:
                    return child
            return None

    board = observation.board
    mark = observation.mark
    
    # If current_state already exists, recycle it based on action taken by opponent
    try:  
        current_state = current_state.choose_child_via_action(
            find_action_taken_by_opponent(board, current_state.board, configuration))
        current_state.parent = None  # make current_state the root node, dereference parents and siblings
        
    except:  # new game or other error in recycling attempt due to Kaggle mechanism
        current_state = State(board, mark,  # This state is considered after the opponent's move
                              configuration, parent=None, is_terminal=False, terminal_score=None, action_taken=None)
   
    # Run MCTS iterations until time limit is reached.
    while time.time() - init_time <= T_max:
        current_state.tree_single_run()
        
    current_state = current_state.choose_play_child()
    return current_state.action_taken

Writing agent_MCTS_MatanTsipory.py


In [6]:
%run agent_MCTS_MatanTsipory.py

## [MCTS w/ Adaptive Playouts](https://www.kaggle.com/code/glazed/monte-carlo-w-adaptive-playouts) - [*glazed*](https://www.kaggle.com/glazed)

In [7]:
%%writefile MCTS_w_Adaptive_Playouts_glazed.py

import time
import numpy as np
from random import choice
from math import sqrt
from numba import njit
from numba import prange

# The following several functions are run with the Numba JIT compiler
# resulting in a dramatic speed increase.

@njit() # Numba function
def win_for_tup_jit(b, tup, I, color):
    # b[] holds the board state.
    # tup[] holds 4 board positions that are in a row.
    # I is one of those positions.
    # Would placing a color piece on I, result in 4 in a row?
    for x in tup:
        if (not ((b[x] == color) or (b[x] == 0))):
            return False
        if (not ((b[x] == color) or (x == I))):
            return False
    return True  

@njit()
def winning_move_jit(b, tupCounts, spaceTups, x_in, color_in):
    # Would placing a color_in piece on I, result in 4 in a row?
    # b[] holds the board state.
    # tupCounts[] and spaceTups[,,] are lookup tables to help examine
    # different groups of 4 positions in a row.
    tup = np.zeros(4, np.int32)
    x = int(x_in)
    color = int(color_in)
    for n in range(tupCounts[x]):# for each tup
        for z in range(4):
            tup[z] = spaceTups [x, n, z]
        if (win_for_tup_jit(b, tup, x, color)):
            return(True)
    return False   

@njit()
def lowest_empty_row_jit(b, j):
    # Return the row for a stone placed in column j.
    # (My coordinates are upside down wrt the contest. )
    r = 6
    c = 7
    for i in range(r): # rows
        x = (i * c) + j
        if (b[x] == 0):
            return i
        # Paronoid check to avoid race conditions if Numba makes this parallel.
        #if ((b[x] == 0) and (i == 0)):
        #    return i
        #if ((b[x] == 0) and (i > 0) and (b[x - c] != 0)):
        #    return i
    return(r)

@njit()
def get_sensible_moves_jit(b, tupCounts, spaceTups, color, otherColor):
    # Return a list of moves worth considering, plus the status of the board.
    # Is there a winning or forced move? Return a list of length one.
    # Also return a status flag.
    # status 0 : tie game, no legal moves
    # status 1 : ongoing game
    # status 2 : Color will win by playing the (single) move in obviousMoves[]
    r = 6 # number of rows
    c = 7 # number of columns
    N = 42 # number of board spaces
    legalMoves = [np.int32(x) for x in range(0)] # weird hack so numba can guess what type is in the list
    for j in range(c):
        i = lowest_empty_row_jit(b, j)
        if (i < r): # legal move
            x = (i * c) + j
            if (winning_move_jit(b, tupCounts, spaceTups, x, color)):
                #print("win")
                obviousMoves = [x]
                return (obviousMoves, 2)
            legalMoves.append(x)
    
    if (len(legalMoves) == 0):
        return (legalMoves, 0)  # tie game
    
    if (len(legalMoves) == 1):
        return (legalMoves, 1)  # ongoing game
    
    for x in legalMoves:
        if (winning_move_jit(b, tupCounts, spaceTups, x, otherColor)):
            #print("forced")
            obviousMoves = [x]
            return (obviousMoves, 1)
    
    lm = legalMoves.copy()
    for x in lm:
        if (x + c < N):
            b[x] = color # temporarily place stone here
            if ((len(legalMoves) > 1) and (winning_move_jit(b, tupCounts, spaceTups, x + c, otherColor))):
                legalMoves.remove(x)
            b[x] = 0 # remove temporarily stone
    return (legalMoves, 1)  # no obvious move


@njit()
def friendly_tupple_jit(b, tup, x, color):
    # tup[] holds 4 board positions that are in a row.
    # x is one of those positions.
    # If there are no othercolor pieces in tup, how many color pieces are there?
    count = np.int32(0)
    nope = False
    if (b[x] != 0):
        return (-1)
    for i in range(4):
        z = tup[i]
        if ((b[z] != color) and (b[z] != 0)):
            nope = True
        if (b[z] == color):
            count += 1
    if (nope): return(-1)
    return count

@njit()
def calcMoveFeatures_jit(b, tupCounts, spaceTups, x, color):
    # calculate features for a move at position x and return in feat[] 
    # tupCounts[] and spaceTups[,,] are lookup tables to help examine
    # different groups of 4 positions in a row.
    c = 7
    r = 6
    feat = np.zeros(22, np.int32)
    tup = np.zeros(4, np.int32)
    #self.feat.fill(0)
    otherColor = 2
    if (color == 2):
        otherColor = 1
    i = x // c
    j =  x % c
    #feat = feat * 0  # clear feat[]
    feat[0] = min(j, (c - 1) - j)  # distance from edge
    tups = tupCounts[x]
    for n in range(tups):
        #tup = self.spaceTups [x, n, :]
        for z in range(4): 
            tup[z] = spaceTups [x, n, z]
        count = friendly_tupple_jit(b, tup, x, color)
        if (count > -1):
            feat[count + 1] += 1
        count = friendly_tupple_jit(b, tup, x, otherColor)
        if (count > 0):
            feat[count + 4] += 1
            
    if (i >= r - 1):
        return feat # we're on the top row, so leave all other features at zero
    I = i + 1 # looking at space above x
    b[x] = color  # temporarily put friendly stone on space x
    xp = (I * c) + j
    tups = tupCounts[xp]
    for n in range(tups):
        #tup = self.spaceTups [xp, n, :]
        for z in range(4):
            tup[z] = spaceTups [xp, n, z]
        count = friendly_tupple_jit(b, tup, xp, color)
        if (count > -1):
            feat[count + 8] += 1
        count = friendly_tupple_jit(b, tup, xp, otherColor)
        if (count > 0):
            feat[count + 11] += 1
        
    b[x] = 0  # remove friendly stone from space x
    
    if (i >= r - 2):
        return feat # we're on the next-to top row, so leave all other features at zero 
    I = i + 2 # looking at space above x
    b[x] = color  # temporarily put friendly stone on space x
    b[xp] = otherColor  # temporarily put enemy stone on space xp
    xpp = (I * c) + j
    tups = tupCounts[xpp]
    for n in range(tups):
        #tup = self.spaceTups [xpp, n, :]
        for z in range(4):
            tup[z] = spaceTups [xpp, n, z]
        count = friendly_tupple_jit(b, tup, xpp, color)
        if (count > -1):
            feat[count + 15] += 1
        count = friendly_tupple_jit(b, tup, xpp, otherColor)
        if (count > 0):
            feat[count + 18] += 1
    
    b[x] = 0  # remove friendly stone from space x
    b[xp] = 0  # remove enemy stone from space xp
    return feat
    
@njit()          
def calc_meta_features_jit(feat, x):
    #calculate meta-features for a move at position x and return in metaFeat[]
    
    # all binary features
    #metaFeat = metaFeat * 0  # clear metaFeat[]
    metaFeat = np.zeros((4 + 6 + (21 * 3)), np.int32)
    #self.metaFeat .fill(0)
    c = 7
    i = x // c
    n = 0
    y = feat [0] # distance from edge -> 4 possibilities, 4 'binary' variables
    metaFeat [y] = 1 # only 1 can be non-zero
    n += 4
    # row -> 6 (essentially boolean) parameters
    metaFeat [n + i] = 1 # only 1 can be non-zero
    n += 6
    for f in range(1, len(feat)):
        if (feat[f] == 0):
            metaFeat[n] = 1
        elif (feat[f] == 1):
            metaFeat[n + 1] = 1
        elif (feat[f] > 1):
            metaFeat[n + 2] = 1
        n += 3
    return metaFeat

@njit()
def linear_move_scores_jit(b, tupCounts, spaceTups, wts, moves, color):
    # For every move in the list, calculate a score and return them in scores[]
    # moves[] holds the list of moves.
    # b[] holds the board state.
    # wts[] holds the linear weights for the features.
    # tupCounts[] and spaceTups[,,] are lookup tables to help examine
    # different groups of 4 positions in a row.
    #min_score = 0.05
    scores = [np.float64(x) for x in range(0)] # weird hack so numba can guess what type belongs in the list
    total = 0.0
    for i in prange(len(moves)):
        x = moves[i]
        feat = calcMoveFeatures_jit(b, tupCounts, spaceTups, x, color) # fills feat
        metaFeat = calc_meta_features_jit(feat, x)         # fills metafeat
        score = np.sum (metaFeat * wts) 
        scores.append(score)
        total += score
    #scores = scores / np.sum(scores)
    for i in range(len(scores)):
        scores[i] /= total
    #for sc in scores:
    #    if (sc < min_score): sc = min_score
    return scores


@njit()  
def choose_linear_move_jit(b, tupCounts, spaceTups, wts, color):
    # Get the sensible_moves[], score them, return the move with the highest score.
    # score is a weighted sum of features.
    #print("choose_linear_move_jit start")
    otherColor = 2
    if (color == 2):
        otherColor = 1
    sensible_moves, status = get_sensible_moves_jit(b, tupCounts, spaceTups, color, otherColor) # also sets self.status
    #print("choose_linear_move_jit calculated")
    if (len(sensible_moves) == 1):   # If it's a win, there will only be one move, so it returns w/ correct status.
        return(sensible_moves[0], status)
    if (len(sensible_moves) == 0):
        return(-1, 0)   # tie  
    scores = linear_move_scores_jit(b, tupCounts, spaceTups, wts, np.array(sensible_moves), color)
    
    #print("choose_linear_move_jit calculated")
    x = sensible_moves[int(np.argmax(np.array(scores)))]
    #print(sensible_moves)
    #print(scores)
    #print(x)
    return (x, 1)


class GAME_MANAGER():
    
    # This class holds the board, the weights for move features and some lookup tables.
    # It has a method that calculates the tables.
    # Note: "tupples" doesn't mean Python tupples.
    
    def __init__(self):
        self.c = 7
        self.r = 6
        self.N = self.c * self.r
        self.K = 4
        self.b = np.zeros((self.N), np.int32)
        self.tupCounts = np.zeros((self.N), np.int32) # how many tupples contain this space
        self.spaceTups = np.zeros((self.N, 16, self.K), np.int32)# all the tupples 
        self.feat = np.zeros(22, np.int32)
        self.metaFeat = np.zeros((4 + 6 + (21 * 3)), np.int32)
        self.precalcTups()  # calculates tupCounts[] and spaceTups[,,]
        self.wts = np.array([0, 3.30366111407672, 6.48699261816764, 16.0570530870234, 0, 6.89154554317436, 12.2291091073301, 14.9610249625214, 8.2764687731129, 5.36422024027117, 14.3516002179586, 2.1021183007845, 0, 0.719857647774551, 2.34487198327812, 11.2057343305379, 0, 27.8825518166704, 43.8228479471015, 0, 1.99103479234425, 0.726840470816211, 0.560352057248385, 4.22057034862125, 6.83487190573809, 0, 17.491336765736, 35.714660516821, 0, 5.87818176004432, 39.6996173910764, 0, 3.6234074451734, 4.49879127350228, 0.806205543215103, 1.71224017973768, 3.8749091066669, 2.53500463742859, 4.460882129649, 1.37971933373037, 44.3252027404174, 29.7283585631117, 0, 0, 2.19363385696125, 1.47116517810174, 11.3140778960712, 2.27101838697622, 0, 1.74611338807736, 6.51927915172379, 0, 2.94225634035759, 0, 3.50453812682987, 0, 0.267382395928564, 1.98693848260618, 0, 1.58478060332119, 4.71565983853622, 0, 17.8551290536231, 7.25762646495066, 5.7077735522044, 1.10042450222865, 0, 8.22587152564711, 4.70559492215376, 0.324579480451124, 16.3108479710915, 8.11547320076288, 0, ]) 
           
    def precalcTups(self):
        # tupCounts[] and spaceTups[,,] are lookup tables to help examine
        # different groups of 4 positions in a row.
        #tupCounts *= 0
        # horizontal
        tup = np.zeros((self.K), np.int32)
        for j in range (0, (self.c - self.K) + 1):
            for i in range(self.r):
                # fill tup[]
                for h in range(self.K):
                    tup[h] = (i * self.c) + j + h
                for h in range(self.K):
                    x = tup [h]
                    n = self.tupCounts [x]
                    for z in range(self.K):
                        self.spaceTups [x, n, z] = tup [z]
                    self.tupCounts [x] = n + 1
        # vertical
        for j in range (self.c):
            for i in range(0, (self.r - self.K) + 1):
                # fill tup[]
                for h in range(self.K):
                    tup[h] = ((i + h) * self.c) + j
                for h in range(self.K):
                    x = tup [h]
                    n = self.tupCounts [x]
                    for z in range(self.K):
                        self.spaceTups [x, n, z] = tup [z]
                    self.tupCounts [x] = n + 1
        # diagonal up, up
        for j in range (0, (self.c - self.K) + 1):
            for i in range(0, (self.r - self.K) + 1):
                # fill tup[]
                for h in range(self.K):
                    tup[h] = ((i + h) * self.c) + j + h
                for h in range(self.K):
                    x = tup [h]
                    n = self.tupCounts [x]
                    for z in range(self.K):
                        self.spaceTups [x, n, z] = tup [z]
                    self.tupCounts [x] = n + 1
        # diagonal something, something...
        for j in range (0, (self.c - self.K) + 1):
            for i in range(self.r-1, self.r - self.K, -1):
                # fill tup[]
                for h in range(self.K):
                    tup[h] = ((i - h) * self.c) + j + h
                for h in range(self.K):
                    x = tup [h]
                    n = self.tupCounts [x]
                    for z in range(self.K):
                        self.spaceTups [x, n, z] = tup [z]
                    self.tupCounts [x] = n + 1
        #print("precalcTups",self.tupCounts)
        return
     
    #@njit()       
    def reset_b(self, B):
        for i in range(len(B)):
            self.b[i] = B[i]


# This class uses all moves as first (AMAF) which is similar to RAVE in MCTS
# There are 42 positions on the board. 
# raveS[] holds the score for each position.
# raveV[] holds the number of visits for each position.
# This class holds methods for UCT-like move selection using RAVE plus policy scores.
class AMAF():
    def __init__(self, N):
        self.N = N
        self.raveS = np.zeros (N, int)  # myColor AMAF
        self.raveV = np.zeros (N, int)
        
    def clearit(self):
        self.raveS.fill(0)
        self.raveV.fill(0)
   
    def amaf_scores(self, moves):
        #return amaf scores for moves 
        scores = []
        for x in moves:
            scores.append( self.raveS[x] / max(1, self.raveV[x]) )
        return(scores)
    
    def PUCT_scores(self, Cpuct, moves, scores):
        if (len(moves) == 0) : print("PUCT_scores BADNESS! length 0")
        if (len(scores) == 0) : print("scores PUCT_scores BADNESS! length 0")
        encounters = 0
        pscores = scores.copy()
        for x in moves:
            encounters += self.raveV[x]
        if (encounters == 0):
            return (pscores)
        for k in range(len(moves)):
            x = moves[k]
            u = self.raveS[x] / max(1, self.raveV[x])
            u += scores[k] * Cpuct * sqrt(encounters) / (self.raveV[x] + 1)
            pscores[k] = u
        return(pscores)
    
    def PUCT_no_policy(self, Cpuct, moves):
        # Returns a move using UCT-like algorithm.
        # Used when no policy scores are avaulable.
        # moves[] holds the moves to be considered.
        if (len(moves) == 0) : print("PUCT_no_policy BADNESS! length 0")
        encounters = 0
        best_score = 0
        best = moves[0]
        for x in moves:
            encounters += self.raveV[x]
        if (encounters == 0):
            return (choice(moves))
        for x in moves:
            u = self.raveS[x] / max(1, self.raveV[x])
            if (u == 2):
                return(x)  # perfect score, so why explore
            u += (1.0 /len(moves) ) * Cpuct * sqrt(encounters) / (self.raveV[x] + 1)
            if (u >= best_score):
                best_score = u
                best = x
        #print("amaf_puct", x)
        return(best)
    
    def PUCT(self, Cpuct, moves, scores):
        # Returns a move using an UCT-like algorithm.
        # scores[] holds the policy scores.
        # moves[] holds the moves to be considered.
        if (len(moves) == 0) : print("PUCT BADNESS! length 0")
        if (len(scores) == 0) : print("scores PUCT BADNESS! length 0")
        encounters = 0
        best_score = 0
        best = moves[0]
        for x in moves:
            encounters += self.raveV[x]
        if (encounters == 0):
            return (choice(moves))
        for k in range(len(moves)):
            x = moves[k]
            u = self.raveS[x] / max(1, self.raveV[x])
            if (u == 2):
                return(x)  # perfect score, so why explore
            u += scores[k] * Cpuct * sqrt(encounters) / (self.raveV[x] + 1)
            if (u >= best_score):
                best_score = u
                best = x
        #print("amaf_puct", x)
        return(best)
    
    
    def reinforce(self, moves, reward):
        # moves holds a list of all the moves (for this color) in one playout.
        for k in moves:
            self.raveV[k] += 1  
            self.raveS[k] += reward


class BRAIN():
    # This class chooses a move using MonteCarlo with adaptive playouts.

    def __init__(self):
        self.gm = GAME_MANAGER() # holds the board and some tables
        self.colorAMAF = AMAF(self.gm.N) # AMAF object for color moves in playouts
        self.otherAMAF = AMAF(self.gm.N) # AMAF object for otherColor moves in playouts
        
     
    def brain_linear_move(self, b, color, V = False):
        # Pick a move just using heuristics. (Not called)
        ts = time.time()
        self.gm.reset_b(b)
        otherColor = 1 + (2 - color)
        moves, status = get_sensible_moves_jit(self.gm.b, self.gm.tupCounts, self.gm.spaceTups, color, otherColor)
        linear_move_scores_jit(self.gm.b, self.gm.tupCounts, self.gm.spaceTups, self.gm.wts, np.array(moves), otherColor) 
       
        x, status = choose_linear_move_jit(self.gm.b, self.gm.tupCounts, self.gm.spaceTups, self.gm.wts, color)
        if (V): print(" JIT tm",time.time() - ts)   
   
        return(x, status)
    
    
    def clearAMAF(self):
        self.colorAMAF.clearit()
        self.otherAMAF.clearit()
        
            
    def MC_adaptive_move(self, root, color, start, time_limit, POLICY):
        # Choose a move using Monte Carlo playouts.
        # First check that there's not only one real choice.
        # Use all time available.
        # Use adaptive playouts.
        # Play move with most visits at the root
        # Only reinforce discretionary moves.
        # if (POLICY): calculate and use heuristic move scores ('priors') in playouts
        
        self.clearAMAF()
        showit = True
        
        CpuctRoot = 7.0 # exploration term for root
        Cpuct = 4.0     # exploration term for all other nodes
        otherColor = 2
        if (color == 2):
            otherColor = 1
        
        self.gm.b = root.copy()   # Set b in game_manager (This is the root board state)
        # Get sensible moves 
        legalMoves, status = get_sensible_moves_jit(self.gm.b, self.gm.tupCounts, self.gm.spaceTups, color, otherColor)
        # If the move is obvious, we return it along with the status
        if (status == 2):
            print("win")
        #    return(legalMoves[0], status)
        if (len(legalMoves) == 1):
            print("forced move")
            return(legalMoves[0], status)
        if (len(legalMoves) == 0):
            print("tie")
            return(-1, 0)   # tie
        
        # There must be 2 or more sensible moves to choose from
        use_policy = POLICY
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        # run MonteCarlo playouts for all 'sensible' moves
        # 
        
        rootMoves = legalMoves.copy()
      
        training_visits = np.zeros(7)
        root_visits = np.zeros(len(rootMoves))
        root_rewards = 0.0
        if (use_policy):
            root_scores = linear_move_scores_jit(self.gm.b, self.gm.tupCounts, self.gm.spaceTups, self.gm.wts, np.array(rootMoves), color) 
        else:
            root_scores = np.ones(len(rootMoves))
            root_scores = root_scores / np.sum(root_scores)
        # run the playouts
        end = time.time()
        reps = 0 
        
        # Run as many playouts as time permits (within reason).
        while ((reps == 0) or ((end-start < time_limit) and (reps < 30000))):
            # Run a single Monte carlo playout - with adaptive playouts using AMAF and priors
            use_policy = POLICY
            reps += 1
            self.gm.b = root.copy() # reset board to root state
            movesColor = [] # moves made by color this playout
            movesOther = [] # moves made by otherColor this playout
            # make a move at the root, a color move
            if (use_policy):
                j = self.colorAMAF.PUCT(CpuctRoot, rootMoves, root_scores)
            else:
                j = self.colorAMAF.PUCT_no_policy(CpuctRoot, rootMoves)
            training_visits[j % 7] += 1 # keep track of starting column
            #print("root move", j)
            self.gm.b[j] = color
            movesColor.append(j)
            cnt = 1
            status = 1
            while (status == 1):
                # ~~~~~~~~~~~~~~~~~~~ otherColor moves ~~~~~~~~~~~~~~~
                moves, status = get_sensible_moves_jit(self.gm.b, self.gm.tupCounts, self.gm.spaceTups, otherColor, color)
                if (status == 2):
                    result = otherColor
                    x = moves[0]
                    self.gm.b[x] = otherColor
                    movesOther.append(x)
                if (status == 0):
                    result = 0
                if (status == 1):
                    if (len(moves) == 1):
                        x = moves[0]
                        #movesOther.append(x) #sort of a forced move so better not to use it
                    else:
                        if (use_policy):
                            scores = linear_move_scores_jit(self.gm.b, self.gm.tupCounts, self.gm.spaceTups, self.gm.wts, np.array(moves), otherColor) 
                            x = self.otherAMAF.PUCT(Cpuct, moves, scores)
                        else:
                            x = self.otherAMAF.PUCT_no_policy(Cpuct, moves)
                        movesOther.append(x)    # use this move to reinforce RAVE
                    self.gm.b[x] = otherColor
                    cnt += 1
                    # ~~~~~~~~~~~~~~~~~~~ color moves ~~~~~~~~~~~~~~~
                    moves, status = get_sensible_moves_jit(self.gm.b, self.gm.tupCounts, self.gm.spaceTups, color, otherColor)
                    if (status == 2):
                        result = color
                        x = moves[0]
                        self.gm.b[x] = color
                        movesColor.append(x)
                    if (status == 0):
                        result = 0
                    if (status == 1):
                        if (len(moves) == 1):
                            x = moves[0]
                            #movesColor.append(x) #sort of a forced move so better not to use it
                        else:
                            if (use_policy):
                                scores = linear_move_scores_jit(self.gm.b, self.gm.tupCounts, self.gm.spaceTups, self.gm.wts, np.array(moves), color) 
                                x = self.colorAMAF.PUCT(Cpuct, moves, scores)
                            else:
                                x = self.colorAMAF.PUCT_no_policy(Cpuct, moves)
                            movesColor.append(x)
                        self.gm.b[x] = color
                        cnt += 1
           
            reward = 0
            if (result == color):
                reward = 2
            elif (result == 0):
                reward = 1
            root_rewards += reward
            # reinforce rave (AMAF)
            self.colorAMAF.reinforce(movesColor, reward)
            self.otherAMAF.reinforce(movesOther, 2 - reward)
            end = time.time()
            if (end-start > time_limit):
                break
            
        #print(scores)
        end = time.time()
        #print("root_scores", root_scores)
        for k in range(len(rootMoves)):   # for display/diagnostics
            x = rootMoves[k]
            root_scores[k] = 100*self.colorAMAF.raveS[x] / max(1, self.colorAMAF.raveV[x])
        #print("root_scores", root_scores)
        j = int(np.argmax(training_visits)) # the column
        i = lowest_empty_row_jit(root, j) # the row
        x = j + i * self.gm.c
        root_rewards /= reps  # average reward for color
        #print (root_visits, 'reps', reps, end-start, x % 7)
        #print (training_visits, 'reps', reps, end-start, x % 7)
        # end of Monte Carlo block
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        #b = root.copy() # reset board to root state
        if (showit): 
            print (root_visits, "root_rewards", root_rewards, 'stone_count', np.sum(root != 0), 'playouts', reps, "***")
            #net_move_scores   (b, rootMoves, color, otherColor, showit = True)
        
        status = 1 # ongoing game
        # return column and status
        return (x, status)  
   
brain = BRAIN() # The object that holds everything


def mirror_top_to_bottom(B):
   # Change board to my (upside down) internal coordinates 
    b = np.zeros((brain.gm.N), int)
    for i in range(6): # rows of b
        for j in range(7):
            x = (i * 7) + j
            mx = ((5 - i) * 7) + j # mirrored top to bottom
            b[mx] = B[x]
    return(b)    


    
def MCTS_wAdaptive_Playouts_(observation, configuration):
    # calls brain to make a move 
    bail_early = False
    start = time.time()
    #The amount of thinking time per move. Add small margin.
    #time_limit = configuration.actTimeout - 0.25
    time_limit = 0.7
    # My coordinates are upside down, so I must flip the input
    b = mirror_top_to_bottom(np.array(observation.board))
    
    
    stone_count = np.sum(b != 0)
    if (stone_count <= 1): time_limit = time_limit / 2
    color = observation.mark 
    
    
    if (stone_count <= 1): print('time_limit', time_limit)
        
        
    rootMove, status = brain.MC_adaptive_move(b, color, start, time_limit, POLICY = True)
    
    
    #x, status = brain.brain_linear_move(b, color) just uses heuristics, no search
    if (bail_early and ((status == 2) or (stone_count > 40))): 
        print('About to win (or tie); cant have that, bailing')
        return(-1)   # Cause an error so as to fail validation and not waste a submission slot.
    
    x = rootMove # x is the board position (one of 42)
    x = x % brain.gm.c  # We must return the column
    
    
    print('time_limit', time_limit, "my duration",time.time() - start, 'x', x)
    return(x)


Writing MCTS_w_Adaptive_Playouts_glazed.py


In [8]:
%run MCTS_w_Adaptive_Playouts_glazed.py

## [Monty Carlo Tree Search](https://www.kaggle.com/code/matant/monte-carlo-tree-search-connectx) - [*James McGuigan*](https://www.kaggle.com/jamesmcguigan)

Monty Carlo Tree Search using an Object Oriented Tree with Numpy Bitboard Bitshifting

In [9]:
%%writefile MontyCarloTreeSearch_JamesMcGuigan.py
#!/usr/bin/env python3

##### 
##### ../../kaggle_compile.py agents/MontyCarlo/MontyCarloPure.py
##### 
##### 2020-08-26 16:45:21+01:00
##### 
##### origin	git@github.com:JamesMcGuigan/ai-games.git (fetch)
##### origin	git@github.com:JamesMcGuigan/ai-games.git (push)
##### 
##### * master 247327a [ahead 6] ConnectX | reduce safety_time to 0.25s
##### 
##### 247327afa97dfaa0c87ea36321e7be3deaa9d8d4
##### 

#####
##### START core/ConnectXBBNN.py
#####

# This is a functional implementation of ConnectX that has been optimized using both numpy and numba

from collections import namedtuple
from typing import List
from typing import Tuple
from typing import Union
from numba import njit, int8, int64

import numba
import numpy as np

# Hardcode for simplicity
# observation   = {'mark': 1, 'board': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}
# configuration = {'columns': 7, 'rows': 6, 'inarow': 4, 'steps': 1000, 'timeout': 8}

bitboard_type = numba.typeof(np.ndarray((2,), dtype=np.int64))
Configuration = namedtuple('configuration', ['rows', 'columns', 'inarow'])
configuration = Configuration(
    rows=6,
    columns=7,
    inarow=4
)



### Conversions

def cast_configuration(configuration):
    return Configuration(
        rows    = configuration.rows,
        columns = configuration.columns,
        inarow  = configuration.inarow
    )


def is_bitboard(bitboard) -> bool:
    if isinstance(bitboard, np.ndarray) and bitboard.dtype == np.int64 and bitboard.shape == (2,):
        return True
    else:
        return False

#@njit
def list_to_bitboard(listboard: Union[np.ndarray,List[int]]) -> np.ndarray:
    # bitboard[0] = played, is a square filled             | 0 = empty, 1 = filled
    # bitboard[1] = player, who's token is this, if filled | 0 = empty, 1 = filled
    bitboard_played = 0  # 42 bit number for if board square has been played
    bitboard_player = 0  # 42 bit number for player 0=p1 1=p2
    if isinstance(listboard, np.ndarray): listboard = listboard.flatten()
    for n in range(len(listboard)):  # prange
        if listboard[n] != 0:
            bitboard_played |= (1 << n)        # is a square filled (0 = empty | 1 = filled)
            if listboard[n] == 2:
                bitboard_player |= (1 << n)    # mark as player 2 square, else assume p1=0 as default
    bitboard = np.array([bitboard_played, bitboard_player], dtype=np.int64)
    return bitboard


@njit(int8[:,:](int64[:]))
def bitboard_to_numpy2d(bitboard: np.ndarray) -> np.ndarray:
    global configuration
    rows    = configuration.rows
    columns = configuration.columns
    size    = rows * columns
    output  = np.zeros((size,), dtype=np.int8)
    for i in range(size):  # prange
        is_played = (bitboard[0] >> i) & 1
        if is_played:
            player = (bitboard[1] >> i) & 1
            output[i] = 1 if player == 0 else 2
    return output.reshape((rows, columns))


### Bitboard Operations

@njit
def empty_bitboard() -> np.ndarray:
    return np.array([0, 0], dtype=np.int64)


def bitboard_from_actions(actions: List[Union[int, Tuple[int]]]) -> np.ndarray:
    bitboard  = empty_bitboard()
    player_id = 1
    for action in actions:
        if isinstance(action, tuple): action, player_id = action
        bitboard  = result_action(bitboard, action, player_id=player_id % 2)
        player_id = next_player_id(player_id)
    return bitboard


@njit
def hash_bitboard( bitboard: np.ndarray ) -> Tuple[int,int]:
    """ Create a tupleised mirror hash, the minimum value of the bitboard and its mirrored reverse """
    if bitboard[0] == 0:
        return ( bitboard[0], bitboard[1] )

    global configuration
    mirror_0 = mirror_bitstring(bitboard[0])
    if bitboard[0] < mirror_0:
        return ( bitboard[0], bitboard[1] )
    else:
        mirror_1 = mirror_bitstring(bitboard[1])
        if bitboard[0] == mirror_0 and bitboard[1] <= mirror_1:
            return ( bitboard[0], bitboard[1] )
        else:
            return ( mirror_0, mirror_1 )


# Use string reverse to create mirror bit lookup table: mirror_bits[ 0100000 ] == 0000010
mirror_bits = np.array([
    int( "".join(reversed(f'{n:07b}')), 2 )
    for n in range(2**configuration.columns)
], dtype=np.int64)

@njit
def mirror_bitstring( bitstring: int ) -> int:
    """ Return the mirror view of the board for hashing:  0100000 -> 0000010 """
    global configuration

    if bitstring == 0:
        return 0  # short-circuit for empty board

    bitsize     = configuration.columns * configuration.rows        # total number of bits to process
    unit_size   = configuration.columns                             # size of each row in bits
    unit_mask   = (1 << unit_size) - 1                              # == 0b1111111 | 0x7f
    offsets     = np.arange(0, bitsize, unit_size, dtype=np.int64)  # == [ 0, 7, 14, 21, 28, 35 ]

    # row_masks   = unit_mask               << offsets  # create bitmasks for each row
    # bits        = (bitstring & row_masks) >> offsets  # extract out the bits for each row
    # stib        = mirror_bits[ bits ]     << offsets  # lookup mirror bits for each row and shift back into position
    # output      = np.sum(stib)                        # np.sum() will bitwise AND the array assuming no overlapping bits

    # This can technically be done as a one liner:
    output = np.sum( mirror_bits[ (bitstring & (unit_mask << offsets)) >> offsets ] << offsets )

    ### Old Loop Implementation
    # output = 0
    # for row in range(configuration.rows):
    #     offset = row * configuration.columns
    #     mask   = unit_mask          << offset
    #     bits   = (bitstring & mask) >> offset
    #     if bits == 0: continue
    #     stib   = mirror_bits[ bits ]
    #     output = output | (stib << offset)

    return int(output)


@njit
def mirror_bitboard( bitboard: np.ndarray ) -> np.ndarray:
    return np.array([
        mirror_bitstring(bitboard[0]),
        mirror_bitstring(bitboard[1]),
    ], dtype=bitboard.dtype)



### Player Id

@njit
def current_player_id( bitboard: np.ndarray ) -> int:
    """ Returns next player to move: 1 = p1, 2 = p2 """
    move_number = get_move_number(bitboard)
    next_player = 1 if move_number % 2 == 0 else 2  # player 1 has the first move on an empty board
    return next_player

def current_player_index( bitboard: np.ndarray ) -> int:
    """ Returns next player to move: 0 = p1, 1 = p2 """
    move_number = get_move_number(bitboard)
    next_player = 0 if move_number % 2 == 0 else 1  # player 1 has the first move on an empty board
    return next_player


@njit(int8(int8))
def next_player_id(player_id: int) -> int:
    # assert player_id in [1,2]
    return 1 if player_id == 2 else 2



### Coordinates

@njit
def index_to_coords(index: int) -> Tuple[int,int]:
    global configuration
    row    = index // configuration.columns
    column = index - row * configuration.columns
    return (row, column)


@njit
def coords_to_index(row: int, column: int) -> int:
    global configuration
    return column + row * configuration.columns



### Moves

@njit(int64[:](int8))
def get_bitcount_mask(size: int = configuration.columns * configuration.rows) -> np.ndarray:
    # return np.array([1 << index for index in range(0, size)], dtype=np.int64)
    return 1 << np.arange(0, size, dtype=np.int64)

# bitcount_mask = get_bitcount_mask()


@njit(int8(int64[:]))
def get_move_number(bitboard: np.ndarray) -> int:
    global configuration
    if bitboard[0] == 0: return 0
    size          = configuration.columns * configuration.rows
    mask_bitcount = get_bitcount_mask(size)
    move_number   = np.count_nonzero(bitboard[0] & mask_bitcount)
    return move_number


mask_board       = (1 << configuration.columns * configuration.rows) - 1
mask_legal_moves = (1 << configuration.columns) - 1

@njit
def has_no_illegal_moves( bitboard: np.ndarray ) -> int:
    """If any the squares on the top row have been played, then there are illegal moves"""
    are_all_moves_legal = ((bitboard[0] & mask_legal_moves) == 0)
    return 1 if are_all_moves_legal else 0


@njit
def has_no_more_moves(bitboard: np.ndarray) -> bool:
    """If all the squares on the top row have been played, then there are no more moves"""
    return bitboard[0] & mask_legal_moves == mask_legal_moves


_is_legal_move_mask  = ((1 << configuration.columns) - 1)
_is_legal_move_cache = np.array([
    [
        int( (bits >> action) & 1 == 0 )
        for action in range(configuration.columns)
    ]
    for bits in range(2**configuration.columns)
], dtype=np.int8)

@njit
def is_legal_move(bitboard: np.ndarray, action: int) -> int:
    bits = bitboard[0] & _is_legal_move_mask   # faster than: int( (bitboard[0] >> action) & 1 == 0 )
    return _is_legal_move_cache[bits, action]  # NOTE: [bits,action] is faster than [bits][action]

#@njit
def get_legal_moves(bitboard: np.ndarray) -> np.ndarray:
    # First 7 bytes represent the top row. Moves are legal if the sky is unplayed
    global configuration
    bits = bitboard[0] & _is_legal_move_mask  # faster than: int( (bitboard[0] >> action) & 1 == 0 )
    if bits == 0:
        return actions  # get_all_moves()
    else:
        return np.array([
            action
            for action in range(configuration.columns)
            if _is_legal_move_cache[bits, action]
        ], dtype=np.int8)


actions = np.array([ action for action in range(configuration.columns) ], dtype=np.int64)
@njit
def get_all_moves() -> np.ndarray:
    # First 7 bytes represent the top row. Moves are legal if the sky is unplayed
    return actions
    # global configuration
    # return np.array([ action for action in range(configuration.columns) ])


@njit
def get_random_move(bitboard: np.ndarray) -> int:
    """ This is slightly quicker than random.choice(get_all_moves())"""
    # assert not has_no_more_moves(bitboard)

    global configuration
    while True:
        action = np.random.randint(0, configuration.columns)
        if is_legal_move(bitboard, action):
            return action



# Actions + Results

@njit
def get_next_index(bitboard: np.ndarray, action: int) -> int:
    global configuration
    # assert is_legal_move(bitboard, action)

    # Start at the ground, and return first row that contains a 0
    for row in range(configuration.rows-1, -1, -1):
        index = action + (row * configuration.columns)
        value = (bitboard[0] >> index) & 1
        if value == 0:
            return index
    return action  # this should never happen - implies not is_legal_move(action)

@njit
def get_next_row(bitboard: np.ndarray, action: int) -> int:
    global configuration
    index = get_next_index(bitboard, action)
    row   = index // configuration.columns
    return row


@njit
def result_action(bitboard: np.ndarray, action: int, player_id: int) -> np.ndarray:
    # assert is_legal_move(bitboard, action)
    index    = get_next_index(bitboard, action)
    mark     = 0 if player_id == 1 else 1
    output = np.array([
        bitboard[0] | 1    << index,
        bitboard[1] | mark << index
    ], dtype=bitboard.dtype)
    return output


### Simulations

#@njit
def run_random_simulation( bitboard: np.ndarray, player_id: int ) -> float:
    """ Returns +1 = victory | 0.5 = draw | 0 = loss """
    move_number = get_move_number(bitboard)
    next_player = 1 if move_number % 2 == 0 else 2  # player 1 has the first move on an empty board
    while not is_gameover(bitboard):
        actions     = get_legal_moves(bitboard)
        action      = np.random.choice(actions)
        bitboard    = result_action(bitboard, action, next_player)
        next_player = next_player_id(next_player)
        # print( bitboard_to_numpy2d(bitboard) )  # DEBUG
    score = get_utility_zero_one(bitboard, player_id)
    return score


### Endgame

@njit(int64[:]())
def get_gameovers() -> np.ndarray:
    """Creates a list of all winning board positions, over 4 directions: horizontal, vertical and 2 diagonals"""
    global configuration

    rows    = configuration.rows
    columns = configuration.columns
    inarow  = configuration.inarow

    gameovers = []

    mask_horizontal  = 0
    mask_vertical    = 0
    mask_diagonal_dl = 0
    mask_diagonal_ul = 0
    for n in range(inarow):  # prange
        mask_horizontal  |= 1 << n
        mask_vertical    |= 1 << n * columns
        mask_diagonal_dl |= 1 << n * columns + n
        mask_diagonal_ul |= 1 << n * columns + (inarow - 1 - n)

    row_inner = rows    - inarow
    col_inner = columns - inarow
    for row in range(rows):  # prange
        for col in range(columns):  # prange
            offset = col + row * columns
            if col <= col_inner:
                gameovers.append( mask_horizontal << offset )
            if row <= row_inner:
                gameovers.append( mask_vertical << offset )
            if col <= col_inner and row <= row_inner:
                gameovers.append( mask_diagonal_dl << offset )
                gameovers.append( mask_diagonal_ul << offset )

    _get_gameovers_cache = np.array(gameovers, dtype=np.int64)
    return _get_gameovers_cache

gameovers = get_gameovers()


@njit
def is_gameover(bitboard: np.ndarray) -> bool:
    if has_no_more_moves(bitboard):  return True
    if get_winner(bitboard) != 0:    return True
    return False


@njit
def get_winner(bitboard: np.ndarray) -> int:
    """ Endgame get_winner: 0 for no get_winner, 1 = player 1, 2 = player 2"""
    global gameovers
    # gameovers = get_gameovers()
    p2_wins = (bitboard[0] &  bitboard[1]) & gameovers == gameovers
    if np.any(p2_wins): return 2
    p1_wins = (bitboard[0] & ~bitboard[1]) & gameovers == gameovers
    if np.any(p1_wins): return 1
    return 0

    # NOTE: above implementation is 2x faster than this original attempt
    # gameovers_played = gameovers[ gameovers & bitboard[0] == gameovers ]  # exclude any unplayed squares
    # if np.any(gameovers_played):                                          # have 4 tokens been played in a row yet
    #     p1_wins = gameovers_played & ~bitboard[1] == gameovers_played
    #     if np.any(p1_wins): return 1
    #     p2_wins = gameovers_played &  bitboard[1] == gameovers_played
    #     if np.any(p2_wins): return 2
    # return 0


### Utility Scores

@njit
def get_utility_one(bitboard: np.ndarray, player_id: int) -> int:
    """ Like get_utility_inf but returns: 1 for victory, -1 for loss, 0 for draw """
    winning_player = get_winner(bitboard)
    if winning_player == 0: return 0
    return 1 if winning_player == player_id else -1


@njit
def get_utility_zero_one(bitboard: np.ndarray, player_id: int) -> float:
    """ Like get_utility_one but returns: 1 for victory, 0 for loss, 0.5 for draw """
    winning_player = get_winner(bitboard)
    if winning_player == 0: return 0.5
    return 1.0 if winning_player == player_id else 0.0


@njit
def get_utility_inf(bitboard: np.ndarray, player_id: int) -> float:
    """ Like get_utility_one but returns: +inf for victory, -inf for loss, 0 for draw """
    winning_player = get_winner(bitboard)
    if winning_player == 0: return 0
    return +np.inf if winning_player == player_id else -np.inf


#####
##### END   core/ConnectXBBNN.py
#####

#####
##### START util/base64_file.py
#####

import base64
import gzip
import os
import pickle
import re
import time
from typing import Any, Union
import humanize

# _base64_file__test_base64_static_import = """
# H4sIAPx9LF8C/2tgri1k0IjgYGBgKCxNLS7JzM8rZIwtZNLwZvBm8mYEkjAI4jFB2KkRbED1iXnF
# 5alFhczeWqV6AEGfwmBHAAAA
# """


def base64_file_varname(filename: str) -> str:
    # ../data/AntColonyTreeSearchNode.pickle.zip.base64 -> _base64_file__AntColonyTreeSearchNode__pickle__zip__base64
    varname = re.sub(r'^.*/',   '',   filename)  # remove directories
    varname = re.sub(r'[.\W]+', '__', varname)   # convert dots and non-ascii to __
    varname = f"_base64_file__{varname}"
    return varname


def base64_file_var_wrap(base64_data: Union[str,bytes], varname: str) -> str:
    return f'{varname} = """\n{base64_data.strip()}\n"""'                    # add varname = """\n\n""" wrapper


def base64_file_var_unwrap(base64_data: str) -> str:
    output = base64_data.strip()
    output = re.sub(r'^\w+ = """|"""$', '', output)  # remove varname = """ """ wrapper
    output = output.strip()
    return output


def base64_file_encode(data: Any) -> str:
    encoded = pickle.dumps(data)
    encoded = gzip.compress(encoded)
    encoded = base64.encodebytes(encoded).decode('utf8').strip()
    return encoded


def base64_file_decode(encoded: str) -> Any:
    data = base64.b64decode(encoded)
    data = gzip.decompress(data)
    data = pickle.loads(data)
    return data


def base64_file_save(data: Any, filename: str, vebose=True) -> float:
    """
        Saves a base64 encoded version of data into filename, with a varname wrapper for importing via kaggle_compile.py
        # Doesn't create/update global variable.
        Returns filesize in bytes
    """
    varname    = base64_file_varname(filename)
    start_time = time.perf_counter()
    try:
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        with open(filename, 'wb') as file:
            encoded = base64_file_encode(data)
            output  = base64_file_var_wrap(encoded, varname)
            output  = output.encode('utf8')
            file.write(output)
            file.close()
        if varname in globals(): globals()[varname] = encoded  # globals not shared between modules, but update for saftey

        filesize = os.path.getsize(filename)
        if vebose:
            time_taken = time.perf_counter() - start_time
            print(f"base64_file_save(): {filename:40s} | {humanize.naturalsize(filesize)} in {time_taken:4.1f}s")
        return filesize
    except Exception as exception:
        pass
    return 0.0


def base64_file_load(filename: str, vebose=True) -> Union[Any,None]:
    """
        Performs a lookup to see if the global variable for this file alread exists
        If not, reads the base64 encoded file from filename, with an optional varname wrapper
        # Doesn't create/update global variable.
        Returns decoded data
    """
    varname    = base64_file_varname(filename)
    start_time = time.perf_counter()
    try:
        # Hard-coding PyTorch weights into a script - https://www.kaggle.com/c/connectx/discussion/126678
        encoded = None

        if varname in globals():
            encoded = globals()[varname]

        if encoded is None and os.path.exists(filename):
            with open(filename, 'rb') as file:
                encoded = file.read().decode('utf8')
                encoded = base64_file_var_unwrap(encoded)
                # globals()[varname] = encoded  # globals are not shared between modules

        if encoded is not None:
            data = base64_file_decode(encoded)

            if vebose:
                filesize = os.path.getsize(filename)
                time_taken = time.perf_counter() - start_time
                print(f"base64_file_load(): {filename:40s} | {humanize.naturalsize(filesize)} in {time_taken:4.1f}s")
            return data
    except Exception as exception:
        print(f'base64_file_load({filename}): Exception:', exception)
    return None


#####
##### END   util/base64_file.py
#####

#####
##### START agents/MontyCarlo/MontyCarloPure.py
#####

# This is a LinkedList implementation of MontyCarlo Tree Search
# Inspired by https://www.kaggle.com/matant/monte-carlo-tree-search-connectx
import atexit
import time
from struct import Struct
from typing import Callable

# from core.ConnectXBBNN import *
# from util.base64_file import base64_file_load
# from util.base64_file import base64_file_save

Hyperparameters = namedtuple('hyperparameters', [])

class MontyCarloNode:
    persist   = True
    save_node = {}                                                        # save_node[cls.__name__] = cls(empty_bitboard(), 1)
    root_nodes: List[Union['MontyCarloNode', None]] = [None, None, None]  # root_nodes[observation.mark]

    def __init__(
            self,
            bitboard:      np.ndarray,
            player_id:     int,
            parent:        Union['MontyCarloNode', None] = None,
            parent_action: Union[int,None]       = None,
            exploration:   float = 1.0,
            **kwargs
    ):
        self.bitboard      = bitboard
        self.player_id     = player_id
        self.next_player   = 3 - player_id

        self.exploration   = exploration
        self.kwargs        = kwargs

        # self.mirror_hash   = hash_bitboard(bitboard)  # BUG: using mirror hashes causes get_best_action() to return invalid moves
        self.legal_moves   = get_legal_moves(bitboard)
        self.is_gameover   = is_gameover(bitboard)
        self.winner        = get_winner(bitboard) if self.is_gameover else 0
        self.utility       = 1 if self.winner == self.player_id else 0  # Scores in range 0-1

        self.parent        = parent
        self.parent_action = parent_action
        self.is_expanded   = False
        self.children: List[Union[MontyCarloNode, None]] = [None for action in get_all_moves()]  # include illegal moves to preserve indexing
        self.total_score   = 0.0
        self.total_visits  = 0
        self.ucb1_score    = self.get_ucb1_score(self)



    def __hash__(self):
        return tuple(self.bitboard)
        # return self.mirror_hash  # BUG: using mirror hashes causes get_best_action() to return invalid moves



    ### Loading and Saving

    @classmethod
    def prune(cls, node: 'MontyCarloNode', min_visits=7, pruned_count=0, total_count=0):
        for n, child in enumerate(node.children):
            if child is None: continue
            if child.total_visits < min_visits:
                pruned_count    += child.total_visits  # excepting terminal states, this equals the number of grandchildren
                total_count     += child.total_visits  # excepting terminal states, this equals the number of grandchildren
                node.children[n] = None
                node.is_expanded = False  # Use def expand(self) to reinitalize state
            else:
                total_count += 1
                pruned_count, total_count = cls.prune(child, min_visits, pruned_count, total_count)
        return pruned_count, total_count


    @classmethod
    def filename(cls):
        return f"data/{cls.__name__}_base64.py"

    @classmethod
    def load(cls):
        filename = cls.filename()
        loaded   = base64_file_load(filename)
        if loaded is not None:
            cls.save_node[cls.__name__] = loaded
            return loaded
        else:
            return None


    @classmethod
    def save(cls) -> Union[str,None]:
        if cls.persist == True and cls.save_node.get(cls.__name__, None) is not None:
            save_node    = cls.save_node[cls.__name__]

            start_time   = time.perf_counter()
            pruned_count, total_count = cls.prune(save_node)  # This reduces a 47MB base64 file down to 5Mb
            print(f'{cls.__name__}.save() - pruned {pruned_count:.0f}/{total_count:.0f} nodes leaving {total_count-pruned_count:.0f} in {time.perf_counter() - start_time:.2f}s')

            filename = cls.filename()
            filesize = base64_file_save(save_node, filename)
            return filename
        return None

    ### Constructors and Lookups

    def create_child( self, action: int ) -> 'MontyCarloNode':
        result = result_action(self.bitboard, action, self.player_id)
        child  = None  # self.find_mirror_child(result, depth=1)  # BUG: using mirror hashes causes get_best_action() to return invalid moves
        if child is None:
            child = self.__class__(
                bitboard      = result,
                player_id     = next_player_id(self.player_id),
                parent        = self,
                parent_action = action,
                exploration   = self.exploration,
                **self.kwargs
            )
        self.children[action] = child
        self.is_expanded      = self._is_expanded()
        if self.is_expanded:
            self.on_expanded()
        return child


    def find_child( self, bitboard: np.array, depth=2 ) -> Union['MontyCarloNode', None]:
        # assert 0 <= depth <= 2

        if depth >= 0:
            if np.all( self.bitboard == bitboard ):
                return self
        if depth >= 1:
            for child in self.children:
                if child is None: continue
                if np.all( child.bitboard == bitboard ):
                    return child
        if depth >= 2:
            # Avoid recursion to prevent duplicate calls to hash_bitboard()
            for child in self.children:
                if child is None: continue
                for grandchild in child.children:
                    if grandchild is None: continue
                    if np.all( grandchild.bitboard == bitboard ):
                        return grandchild
        return None

    # # BUG: using mirror hashes causes get_best_action() to return invalid moves
    # def find_mirror_child( self, bitboard: np.array, depth=2 ) -> Union['MontyCarloNode', None]:
    #     # assert 0 <= depth <= 2
    #     mirror_hash = hash_bitboard(bitboard)
    #
    #     if depth >= 0:
    #         if self.mirror_hash == mirror_hash:
    #             return self
    #     if depth >= 1:
    #         for child in self.children:
    #             if child is None: continue
    #             if child.mirror_hash == mirror_hash:
    #                 return child
    #     if depth >= 2:
    #         # Avoid recursion to prevent duplicate calls to hash_bitboard()
    #         for child in self.children:
    #             if child is None: continue
    #             for grandchild in child.children:
    #                 if grandchild is None: continue
    #                 if grandchild.mirror_hash == mirror_hash:
    #                     return grandchild
    #     return None



    ### Properties

    def _is_expanded(self) -> bool:
        is_expanded = True
        for action in self.legal_moves:
            if self.children[action] is None:
                is_expanded = False
                break
        return is_expanded


    def get_unexpanded(self) -> List[int]:
        return [
            action
            for action in self.legal_moves
            if self.children[action] is None
        ]


    ### Action Selection

    def get_best_action(self) -> int:
        scores = [
            self.children[action].total_score
            if self.children[action] is not None else 0
            for action in self.legal_moves
        ]
        index  = np.argmax(scores)
        action = self.legal_moves[index]
        return action


    def get_exploration_action(self) -> int:
        scores = [
            self.children[action].ucb1_score
            if self.children[action] is not None else 0
            for action in self.legal_moves
        ]
        index  = np.argmax(scores)
        action = self.legal_moves[index]
        return action



    ### Scores

    def get_ucb1_score(self, node: 'MontyCarloNode') -> float:
        if node is None or node.total_visits == 0:
            return np.inf
        else:
            score = node.total_score / node.total_visits
            if node.parent is not None and node.parent.total_visits > 0:
                score += (
                    node.exploration * np.sqrt(2)
                    * np.log(node.parent.total_visits) / node.total_visits
                )
            return score


    @staticmethod
    def opponents_score(score: float):
        # assert 0 <= score <= 1
        return 1 - score



    ### Training and Backpropagation

    def single_run(self):
        if self.is_gameover:
            self.backpropagate(self.utility)
        elif not self.is_expanded:
            child = self.expand()
            score = child.simulate()    # score from the perspective of the other player
            child.backpropagate(score)
        else:
            # Recurse down tree, until a gameover or not expanded node is found
            action = self.get_exploration_action()
            child  = self.children[action]
            child.single_run()


    def expand(self) -> 'MontyCarloNode':
        # assert not self.is_gameover
        # assert not self.is_expanded

        unexpanded = self.get_unexpanded()
        # assert len(unexpanded)

        action     = np.random.choice(unexpanded)
        child      = self.create_child(action)
        return child

    def on_expanded(self) -> None:
        # Callback placeholder for any subclass hooks
        pass

    def simulate(self) -> float:
        return run_random_simulation(self.bitboard, self.player_id)


    def backpropagate(self, score: float):
        # child.simulate()  returns score for the player 2
        # child.total_score is accessed via the parent node, so score on this node is from the perspective of player 1
        node = self
        while node is not None:
            score = self.opponents_score(score)
            node.total_score  += score
            node.total_visits += 1
            node = node.parent      # when we reach the root: node.parent == None which terminates

        # get_ucb1_score() gets called 4x less often if we cache the value after backpropagation
        # get_ucb1_score() depends on parent.total_visits, so needs to be called after updating scores
        node = self
        while node is not None:
            node.ucb1_score = node.get_ucb1_score(node)
            node = node.parent      # when we reach the root: node.parent == None which terminates



    ### Agent
    @classmethod
    def agent(cls, **kwargs) -> Callable[[Struct, Struct],int]:
        def kaggle_agent( observation: Struct, _configuration_: Struct ):
            first_move_time = 0
            safety_time     = kwargs.get('safety_time', 0.25)
            start_time      = time.perf_counter()
            # configuration   = cast_configuration(_configuration_)

            player_id     = int(observation.mark)
            listboard     = np.array(observation.board, dtype=np.int8)
            bitboard      = list_to_bitboard(listboard)
            move_number   = get_move_number(bitboard)
            is_first_move = int(move_number < 2)
          # endtime       = start_time + _configuration_.timeout - safety_time - (first_move_time * is_first_move)
            endtime       = start_time +            1.25          - safety_time - (first_move_time * is_first_move)
            
            if cls.persist == True and cls.save_node.get(cls.__name__, None) is None:
                atexit.register(cls.save)
                cls.save_node[cls.__name__] = cls.load() or cls(empty_bitboard(), player_id=1)
                cls.root_nodes[1] = cls.root_nodes[2] = cls.save_node[cls.__name__]  # implement shared state

            root_node = cls.root_nodes[player_id]
            if root_node is None or root_node.find_child(bitboard, depth=2) is None:
                root_node = cls.root_nodes[player_id] = cls(
                    bitboard      = bitboard,
                    player_id     = player_id,
                    parent        = None,
                    # configuration = configuration,
                    **kwargs
                )
            else:
                root_node = cls.root_nodes[player_id] = cls.root_nodes[player_id].find_child(bitboard)
            # assert root_node is not None

            count = 0
            while time.perf_counter() < endtime:
                count += 1
                root_node.single_run()

            action     = root_node.get_best_action()
            time_taken = time.perf_counter() - start_time
            print(f'{cls.__name__}: p{player_id} action = {action} after {count} simulations in {time_taken:.3f}s')
            return int(action)

        kaggle_agent.__name__ = cls.__name__
        return kaggle_agent

def MontyCarloPure(**kwargs):
    # observation   = {'mark': 1, 'board': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}
    # configuration = {'columns': 7, 'rows': 6, 'inarow': 4, 'steps': 1000, 'timeout': 8}
    def MontyCarloPure(observation: Struct, configuration: Struct) -> int:
        return MontyCarloNode.agent(**kwargs)(observation, configuration)
    return MontyCarloPure

def MontyCarlo_JamesMcGuigan(observation, configuration):
    return MontyCarloPure()(observation, configuration)


#####
##### END   agents/MontyCarlo/MontyCarloPure.py
#####

##### 
##### ../../kaggle_compile.py agents/MontyCarlo/MontyCarloPure.py
##### 
##### 2020-08-26 16:45:21+01:00
##### 
##### origin	git@github.com:JamesMcGuigan/ai-games.git (fetch)
##### origin	git@github.com:JamesMcGuigan/ai-games.git (push)
##### 
##### * master 247327a [ahead 6] ConnectX | reduce safety_time to 0.25s
##### 
##### 247327afa97dfaa0c87ea36321e7be3deaa9d8d4
##### 


Writing MontyCarloTreeSearch_JamesMcGuigan.py


In [10]:
%run MontyCarloTreeSearch_JamesMcGuigan.py

## [MCTS Bitboard + Bitsquares Heuristic](https://www.kaggle.com/code/jamesmcguigan/connectx-mcts-bitboard-bitsquares-heuristic) - [*James McGuigan*](https://www.kaggle.com/jamesmcguigan)

In [11]:
%%writefile MCTS_Bitboard_Bitsquares_Heuristic_JamesMcGuigan.py
#!/usr/bin/env python3

##### 
##### /ai-games/kaggle_compile.py agents/MontyCarlo/MontyCarloBitsquares.py
##### 
##### 2024-01-28 08:30:57+00:00
##### 
##### origin	https://github.com/JamesMcGuigan/ai-games.git (fetch)
##### origin	https://github.com/JamesMcGuigan/ai-games.git (push)
##### 
##### * master eac436d Revert: Rock Paper Scissors | Random Seed Search | tweak unit tests
##### 
##### eac436d23e03624c2245838138e9608cb13d2f1f
##### 

#####
##### START core/ConnectXBBNN.py
#####

# This is a functional implementation of ConnectX that has been optimized using both numpy and numba

from collections import namedtuple
from typing import List
from typing import Tuple
from typing import Union

import numba
import numpy as np

# Hardcode for simplicity
# observation   = {'mark': 1, 'board': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}
# configuration = {'columns': 7, 'rows': 6, 'inarow': 4, 'steps': 1000, 'timeout': 8}

bitboard_type = numba.typeof(np.ndarray((2,), dtype=np.int64))
Configuration = namedtuple('configuration', ['rows', 'columns', 'inarow'])
configuration = Configuration(
    rows=6,
    columns=7,
    inarow=4
)



### Conversions

def cast_configuration(configuration):
    return Configuration(
        rows    = configuration.rows,
        columns = configuration.columns,
        inarow  = configuration.inarow
    )


def is_bitboard(bitboard) -> bool:
    if isinstance(bitboard, np.ndarray) and bitboard.dtype == np.int64 and bitboard.shape == (2,):
        return True
    else:
        return False

#@njit
def list_to_bitboard(listboard: Union[np.ndarray,List[int]]) -> np.ndarray:
    # bitboard[0] = played, is a square filled             | 0 = empty, 1 = filled
    # bitboard[1] = player, who's token is this, if filled | 0 = empty, 1 = filled
    bitboard_played = 0  # 42 bit number for if board square has been played
    bitboard_player = 0  # 42 bit number for player 0=p1 1=p2
    if isinstance(listboard, np.ndarray): listboard = listboard.flatten()
    for n in range(len(listboard)):  # prange
        if listboard[n] != 0:
            bitboard_played |= (1 << n)        # is a square filled (0 = empty | 1 = filled)
            if listboard[n] == 2:
                bitboard_player |= (1 << n)    # mark as player 2 square, else assume p1=0 as default
    bitboard = np.array([bitboard_played, bitboard_player], dtype=np.int64)
    return bitboard


#@njit(int8[:,:](int64[:]))
def bitboard_to_numpy2d(bitboard: np.ndarray) -> np.ndarray:
    global configuration
    rows    = configuration.rows
    columns = configuration.columns
    size    = rows * columns
    output  = np.zeros((size,), dtype=np.int8)
    for i in range(size):  # prange
        is_played = (bitboard[0] >> i) & 1
        if is_played:
            player = (bitboard[1] >> i) & 1
            output[i] = 1 if player == 0 else 2
    return output.reshape((rows, columns))


### Bitboard Operations

#@njit
def empty_bitboard() -> np.ndarray:
    return np.array([0, 0], dtype=np.int64)


def bitboard_from_actions(actions: List[Union[int, Tuple[int]]]) -> np.ndarray:
    bitboard  = empty_bitboard()
    player_id = 1
    for action in actions:
        if isinstance(action, tuple): action, player_id = action
        bitboard  = result_action(bitboard, action, player_id=player_id % 2)
        player_id = next_player_id(player_id)
    return bitboard


#@njit
def hash_bitboard( bitboard: np.ndarray ) -> Tuple[int,int]:
    """ Create a tupleised mirror hash, the minimum value of the bitboard and its mirrored reverse """
    if bitboard[0] == 0:
        return ( bitboard[0], bitboard[1] )

    global configuration
    mirror_0 = mirror_bitstring(bitboard[0])
    if bitboard[0] < mirror_0:
        return ( bitboard[0], bitboard[1] )
    else:
        mirror_1 = mirror_bitstring(bitboard[1])
        if bitboard[0] == mirror_0 and bitboard[1] <= mirror_1:
            return ( bitboard[0], bitboard[1] )
        else:
            return ( mirror_0, mirror_1 )


# Use string reverse to create mirror bit lookup table: mirror_bits[ 0100000 ] == 0000010
mirror_bits = np.array([
    int( "".join(reversed(f'{n:07b}')), 2 )
    for n in range(2**configuration.columns)
], dtype=np.int64)

#@njit
def mirror_bitstring( bitstring: int ) -> int:
    """ Return the mirror view of the board for hashing:  0100000 -> 0000010 """
    global configuration

    if bitstring == 0:
        return 0  # short-circuit for empty board

    bitsize     = configuration.columns * configuration.rows        # total number of bits to process
    unit_size   = configuration.columns                             # size of each row in bits
    unit_mask   = (1 << unit_size) - 1                              # == 0b1111111 | 0x7f
    offsets     = np.arange(0, bitsize, unit_size, dtype=np.int64)  # == [ 0, 7, 14, 21, 28, 35 ]

    # row_masks   = unit_mask               << offsets  # create bitmasks for each row
    # bits        = (bitstring & row_masks) >> offsets  # extract out the bits for each row
    # stib        = mirror_bits[ bits ]     << offsets  # lookup mirror bits for each row and shift back into position
    # output      = np.sum(stib)                        # np.sum() will bitwise AND the array assuming no overlapping bits

    # This can technically be done as a one liner:
    output = np.sum( mirror_bits[ (bitstring & (unit_mask << offsets)) >> offsets ] << offsets )

    ### Old Loop Implementation
    # output = 0
    # for row in range(configuration.rows):
    #     offset = row * configuration.columns
    #     mask   = unit_mask          << offset
    #     bits   = (bitstring & mask) >> offset
    #     if bits == 0: continue
    #     stib   = mirror_bits[ bits ]
    #     output = output | (stib << offset)

    return int(output)


#@njit
def mirror_bitboard( bitboard: np.ndarray ) -> np.ndarray:
    return np.array([
        mirror_bitstring(bitboard[0]),
        mirror_bitstring(bitboard[1]),
    ], dtype=bitboard.dtype)



### Player Id

#@njit
def current_player_id( bitboard: np.ndarray ) -> int:
    """ Returns next player to move: 1 = p1, 2 = p2 """
    move_number = get_move_number(bitboard)
    next_player = 1 if move_number % 2 == 0 else 2  # player 1 has the first move on an empty board
    return next_player

def current_player_index( bitboard: np.ndarray ) -> int:
    """ Returns next player to move: 0 = p1, 1 = p2 """
    move_number = get_move_number(bitboard)
    next_player = 0 if move_number % 2 == 0 else 1  # player 1 has the first move on an empty board
    return next_player


#@njit(int8(int8))
def next_player_id(player_id: int) -> int:
    # assert player_id in [1,2]
    return 1 if player_id == 2 else 2



### Coordinates

#@njit
def index_to_coords(index: int) -> Tuple[int,int]:
    global configuration
    row    = index // configuration.columns
    column = index - row * configuration.columns
    return (row, column)


#@njit
def coords_to_index(row: int, column: int) -> int:
    global configuration
    return column + row * configuration.columns



### Moves

#@njit(int64[:](int8))
def get_bitcount_mask(size: int = configuration.columns * configuration.rows) -> np.ndarray:
    # return np.array([1 << index for index in range(0, size)], dtype=np.int64)
    return 1 << np.arange(0, size, dtype=np.int64)

bitcount_mask = get_bitcount_mask()


#@njit(int8(int64[:]))
def get_move_number(bitboard: np.ndarray) -> int:
    global configuration
    if bitboard[0] == 0: return 0
    size          = configuration.columns * configuration.rows
    mask_bitcount = get_bitcount_mask(size)
    move_number   = np.count_nonzero(bitboard[0] & mask_bitcount)
    return move_number


mask_board       = (1 << configuration.columns * configuration.rows) - 1
mask_legal_moves = (1 << configuration.columns) - 1

#@njit
def has_no_illegal_moves( bitboard: np.ndarray ) -> int:
    """If any the squares on the top row have been played, then there are illegal moves"""
    are_all_moves_legal = ((bitboard[0] & mask_legal_moves) == 0)
    return 1 if are_all_moves_legal else 0


#@njit
def has_no_more_moves(bitboard: np.ndarray) -> bool:
    """If all the squares on the top row have been played, then there are no more moves"""
    return bitboard[0] & mask_legal_moves == mask_legal_moves


_is_legal_move_mask  = ((1 << configuration.columns) - 1)
_is_legal_move_cache = np.array([
    [
        int( (bits >> action) & 1 == 0 )
        for action in range(configuration.columns)
    ]
    for bits in range(2**configuration.columns)
], dtype=np.int8)

#@njit
def is_legal_move(bitboard: np.ndarray, action: int) -> int:
    bits = bitboard[0] & _is_legal_move_mask   # faster than: int( (bitboard[0] >> action) & 1 == 0 )
    return _is_legal_move_cache[bits, action]  # NOTE: [bits,action] is faster than [bits][action]

#@njit
def get_legal_moves(bitboard: np.ndarray) -> np.ndarray:
    # First 7 bytes represent the top row. Moves are legal if the sky is unplayed
    global configuration
    bits = bitboard[0] & _is_legal_move_mask  # faster than: int( (bitboard[0] >> action) & 1 == 0 )
    if bits == 0:
        return actions  # get_all_moves()
    else:
        return np.array([
            action
            for action in range(configuration.columns)
            if _is_legal_move_cache[bits, action]
        ], dtype=np.int8)


actions = np.array([ action for action in range(configuration.columns) ], dtype=np.int64)
#@njit
def get_all_moves() -> np.ndarray:
    # First 7 bytes represent the top row. Moves are legal if the sky is unplayed
    return actions
    # global configuration
    # return np.array([ action for action in range(configuration.columns) ])


#@njit
def get_random_move(bitboard: np.ndarray) -> int:
    """ This is slightly quicker than random.choice(get_all_moves())"""
    # assert not has_no_more_moves(bitboard)

    global configuration
    while True:
        action = np.random.randint(0, configuration.columns)
        if is_legal_move(bitboard, action):
            return action

#@njit
def get_random_draw_move(bitboard: np.ndarray) -> int:
    # Get a random move, but deliberately don't play any winning moves to generate has_no_move_moves() endgames
    # Statistically, only 1 in 7 games generated this way are draws
    actions   = get_legal_moves(bitboard)
    player_id = current_player_id(bitboard)
    while len(actions):
        action = np.random.choice(actions)
        result = result_action(bitboard, action, player_id)
        if get_winner(result):
            actions = np.delete(actions, np.where(actions == action))
        else:
            return action
    return np.random.choice(get_legal_moves(bitboard))  # winning is unavoidable


# Actions + Results

#@njit
def get_next_index(bitboard: np.ndarray, action: int) -> int:
    global configuration
    # assert is_legal_move(bitboard, action)

    # Start at the ground, and return first row that contains a 0
    for row in range(configuration.rows-1, -1, -1):
        index = action + (row * configuration.columns)
        value = (bitboard[0] >> index) & 1
        if value == 0:
            return index
    return action  # this should never happen - implies not is_legal_move(action)

#@njit
def get_next_row(bitboard: np.ndarray, action: int) -> int:
    global configuration
    index = get_next_index(bitboard, action)
    row   = index // configuration.columns
    return row


#@njit
def result_action(bitboard: np.ndarray, action: int, player_id: int) -> np.ndarray:
    # assert is_legal_move(bitboard, action)
    index    = get_next_index(bitboard, action)
    mark     = 0 if player_id == 1 else 1
    output = np.array([
        bitboard[0] | 1    << index,
        bitboard[1] | mark << index
    ], dtype=bitboard.dtype)
    return output


### Simulations

#@njit
def run_random_simulation( bitboard: np.ndarray, player_id: int ) -> float:
    """ Returns +1 = victory | 0.5 = draw | 0 = loss """
    move_number = get_move_number(bitboard)
    next_player = 1 if move_number % 2 == 0 else 2  # player 1 has the first move on an empty board
    while not is_gameover(bitboard):
        actions     = get_legal_moves(bitboard)
        action      = np.random.choice(actions)
        bitboard    = result_action(bitboard, action, next_player)
        next_player = next_player_id(next_player)
        # print( bitboard_to_numpy2d(bitboard) )  # DEBUG
    score = get_utility_zero_one(bitboard, player_id)
    return score


### Endgame

#@njit(int64[:]())
def get_gameovers() -> np.ndarray:
    """Creates a list of all winning board positions, over 4 directions: horizontal, vertical and 2 diagonals"""
    global configuration

    rows    = configuration.rows
    columns = configuration.columns
    inarow  = configuration.inarow

    gameovers = []

    mask_horizontal  = 0
    mask_vertical    = 0
    mask_diagonal_dl = 0
    mask_diagonal_ul = 0
    for n in range(inarow):  # prange
        mask_horizontal  |= 1 << n
        mask_vertical    |= 1 << n * columns
        mask_diagonal_dl |= 1 << n * columns + n
        mask_diagonal_ul |= 1 << n * columns + (inarow - 1 - n)

    row_inner = rows    - inarow
    col_inner = columns - inarow
    for row in range(rows):  # prange
        for col in range(columns):  # prange
            offset = col + row * columns
            if col <= col_inner:
                gameovers.append( mask_horizontal << offset )
            if row <= row_inner:
                gameovers.append( mask_vertical << offset )
            if col <= col_inner and row <= row_inner:
                gameovers.append( mask_diagonal_dl << offset )
                gameovers.append( mask_diagonal_ul << offset )

    _get_gameovers_cache = np.array(gameovers, dtype=np.int64)
    return _get_gameovers_cache

gameovers = get_gameovers()


#@njit
def is_gameover(bitboard: np.ndarray) -> bool:
    if has_no_more_moves(bitboard):  return True
    if get_winner(bitboard) != 0:    return True
    return False


#@njit
def get_winner(bitboard: np.ndarray) -> int:
    """ Endgame get_winner: 0 for no get_winner, 1 = player 1, 2 = player 2"""
    global gameovers
    # gameovers = get_gameovers()
    p2_wins = (bitboard[0] &  bitboard[1]) & gameovers == gameovers
    if np.any(p2_wins): return 2
    p1_wins = (bitboard[0] & ~bitboard[1]) & gameovers == gameovers
    if np.any(p1_wins): return 1
    return 0

    # NOTE: above implementation is 2x faster than this original attempt
    # gameovers_played = gameovers[ gameovers & bitboard[0] == gameovers ]  # exclude any unplayed squares
    # if np.any(gameovers_played):                                          # have 4 tokens been played in a row yet
    #     p1_wins = gameovers_played & ~bitboard[1] == gameovers_played
    #     if np.any(p1_wins): return 1
    #     p2_wins = gameovers_played &  bitboard[1] == gameovers_played
    #     if np.any(p2_wins): return 2
    # return 0


### Utility Scores

#@njit
def get_utility_one(bitboard: np.ndarray, player_id: int) -> int:
    """ Like get_utility_inf but returns: 1 for victory, -1 for loss, 0 for draw """
    winning_player = get_winner(bitboard)
    if winning_player == 0: return 0
    return 1 if winning_player == player_id else -1


#@njit
def get_utility_zero_one(bitboard: np.ndarray, player_id: int) -> float:
    """ Like get_utility_one but returns: 1 for victory, 0 for loss, 0.5 for draw """
    winning_player = get_winner(bitboard)
    if winning_player == 0: return 0.5
    return 1.0 if winning_player == player_id else 0.0


#@njit
def get_utility_inf(bitboard: np.ndarray, player_id: int) -> float:
    """ Like get_utility_one but returns: +inf for victory, -inf for loss, 0 for draw """
    winning_player = get_winner(bitboard)
    if winning_player == 0: return 0
    return +np.inf if winning_player == player_id else -np.inf


#####
##### END   core/ConnectXBBNN.py
#####

#####
##### START util/sigmoid.py
#####

import numpy as np

def sigmoid(self, value: float):
    # self.sigmoid_width == 2 means a heuristic score of +-2 will return +-0.73
    # 1 / (1 + math.exp(-(+np.inf))) == 1.0
    # 1 / (1 + math.exp(-(4.0)))     == 0.99
    # 1 / (1 + math.exp(-(4.0)))     == 0.98
    # 1 / (1 + math.exp(-(3.0)))     == 0.95
    # 1 / (1 + math.exp(-(2.0)))     == 0.88
    # 1 / (1 + math.exp(-(1.0)))     == 0.73
    # 1 / (1 + math.exp(-(0.5)))     == 0.62
    # 1 / (1 + math.exp(-(0.0)))     == 0.5
    # 1 / (1 + math.exp(-(-np.inf))) == 0.0
    return 1 / (1 + np.exp(-value))


def scaled_sigmoid(value: float, sigmoid_width: float, sigmoid_height: float = 1.0) -> float:
    # sigmoid_width  == 2 means a heuristic score of +-2 will return +-0.73
    # sigmoid_height is used to distinguish between gameover() == +-1 and heuristic values
    return sigmoid_height * (1 / (1 + np.exp(-value / sigmoid_width))) if sigmoid_width else 0


#####
##### END   util/sigmoid.py
#####

#####
##### START util/base64_file.py
#####

import base64
import gzip
import os
import re
import time
from typing import Any
from typing import Union

import dill
import humanize


# _base64_file__test_base64_static_import = """
# H4sIAPx9LF8C/2tgri1k0IjgYGBgKCxNLS7JzM8rZIwtZNLwZvBm8mYEkjAI4jFB2KkRbED1iXnF
# 5alFhczeWqV6AEGfwmBHAAAA
# """


def base64_file_varname(filename: str) -> str:
    # ../data/AntColonyTreeSearchNode.dill.zip.base64 -> _base64_file__AntColonyTreeSearchNode__dill__zip__base64
    varname = re.sub(r'^.*/',   '',   filename)  # remove directories
    varname = re.sub(r'[.\W]+', '__', varname)   # convert dots and non-ascii to __
    varname = f"_base64_file__{varname}"
    return varname


def base64_file_var_wrap(base64_data: Union[str,bytes], varname: str) -> str:
    return f'{varname} = """\n{base64_data.strip()}\n"""'                    # add varname = """\n\n""" wrapper


def base64_file_var_unwrap(base64_data: str) -> str:
    output = base64_data.strip()
    output = re.sub(r'^\w+ = """|"""$', '', output)  # remove varname = """ """ wrapper
    output = output.strip()
    return output


def base64_file_encode(data: Any) -> str:
    encoded = dill.dumps(data)
    encoded = gzip.compress(encoded)
    encoded = base64.encodebytes(encoded).decode('utf8').strip()
    return encoded


def base64_file_decode(encoded: str) -> Any:
    data = base64.b64decode(encoded)
    data = gzip.decompress(data)
    data = dill.loads(data)
    return data


def base64_file_save(data: Any, filename: str, vebose=True) -> float:
    """
        Saves a base64 encoded version of data into filename, with a varname wrapper for importing via kaggle_compile.py
        # Doesn't create/update global variable.
        Returns filesize in bytes
    """
    varname    = base64_file_varname(filename)
    start_time = time.perf_counter()
    try:
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        with open(filename, 'wb') as file:
            encoded = base64_file_encode(data)
            output  = base64_file_var_wrap(encoded, varname)
            output  = output.encode('utf8')
            file.write(output)
            file.close()
        if varname in globals(): globals()[varname] = encoded  # globals not shared between modules, but update for saftey

        filesize = os.path.getsize(filename)
        if vebose:
            time_taken = time.perf_counter() - start_time
            print(f"base64_file_save(): {filename:40s} | {humanize.naturalsize(filesize)} in {time_taken:4.1f}s")
        return filesize
    except Exception as exception:
        pass
    return 0.0


def base64_file_load(filename: str, vebose=True) -> Union[Any,None]:
    """
        Performs a lookup to see if the global variable for this file alread exists
        If not, reads the base64 encoded file from filename, with an optional varname wrapper
        # Doesn't create/update global variable.
        Returns decoded data
    """
    varname    = base64_file_varname(filename)
    start_time = time.perf_counter()
    try:
        # Hard-coding PyTorch weights into a script - https://www.kaggle.com/c/connectx/discussion/126678
        encoded = None

        if varname in globals():
            encoded = globals()[varname]

        if encoded is None and os.path.exists(filename):
            with open(filename, 'rb') as file:
                encoded = file.read().decode('utf8')
                encoded = base64_file_var_unwrap(encoded)
                # globals()[varname] = encoded  # globals are not shared between modules

        if encoded is not None:
            data = base64_file_decode(encoded)

            if vebose:
                filesize = os.path.getsize(filename)
                time_taken = time.perf_counter() - start_time
                print(f"base64_file_load(): {filename:40s} | {humanize.naturalsize(filesize)} in {time_taken:4.1f}s")
            return data
    except Exception as exception:
        print(f'base64_file_load({filename}): Exception:', exception)
    return None


#####
##### END   util/base64_file.py
#####

#####
##### START agents/MontyCarlo/MontyCarloPure.py
#####

# This is a LinkedList implementation of MontyCarlo Tree Search
# Inspired by https://www.kaggle.com/matant/monte-carlo-tree-search-connectx
import atexit
import time
from struct import Struct
from typing import Callable

# from core.ConnectXBBNN import *
# from util.base64_file import base64_file_load
# from util.base64_file import base64_file_save

Hyperparameters = namedtuple('hyperparameters', [])

class MontyCarloNode:
    persist   = True
    save_node = None
    root_nodes: List[Union['MontyCarloNode', None]] = [None, None, None]  # root_nodes[observation.mark]

    # This ensures we have unique instances of cls.save_node, cls.root_nodes in case multiple subclasses are running simultaniously
    @classmethod
    def init_class(cls):
        # noinspection PyUnresolvedReferences
        # create a new cls.save_node and cls.root_nodes for each class
        for parentclass in cls.__mro__:  # https://stackoverflow.com/questions/2611892/how-to-get-the-parents-of-a-python-class
            if cls is parentclass: continue
            if ( cls.save_node  is getattr(parentclass, 'save_node',  None)
              or cls.root_nodes is getattr(parentclass, 'root_nodes', None)
            ):
                cls.save_node  = None
                cls.root_nodes = [None, None, None]
                break


    def __init__(
            self,
            bitboard:      np.ndarray,
            player_id:     int,
            parent:        Union['MontyCarloNode', None] = None,
            parent_action: Union[int,None]       = None,
            exploration:   float = 1.0,
            **kwargs
    ):
        self.bitboard      = bitboard
        self.player_id     = player_id
        self.next_player   = 3 - player_id

        self.exploration   = exploration
        self.kwargs        = kwargs

        # self.mirror_hash   = hash_bitboard(bitboard)  # BUG: using mirror hashes causes get_best_action() to return invalid moves
        self.legal_moves   = get_legal_moves(bitboard)
        self.is_gameover   = is_gameover(bitboard)
        self.winner        = get_winner(bitboard) if self.is_gameover else 0
        self.utility       = 1 if self.winner == self.player_id else 0  # Scores in range 0-1

        self.parent        = parent
        self.parent_action = parent_action
        self.is_expanded   = False
        self.children: List[Union[MontyCarloNode, None]] = [None for action in get_all_moves()]  # include illegal moves to preserve indexing
        self.total_score   = 0.0
        self.total_visits  = 0
        self.ucb1_score    = self.get_ucb1_score(self)



    def __hash__(self):
        return tuple(self.bitboard)
        # return self.mirror_hash  # BUG: using mirror hashes causes get_best_action() to return invalid moves



    ### Loading and Saving

    @classmethod
    def prune(cls, node: 'MontyCarloNode', min_visits=7, pruned_count=0, total_count=0):
        for n, child in enumerate(node.children):
            if child is None: continue
            if child.total_visits < min_visits:
                pruned_count    += child.total_visits  # excepting terminal states, this equals the number of grandchildren
                total_count     += child.total_visits  # excepting terminal states, this equals the number of grandchildren
                node.children[n] = None
                node.is_expanded = False  # Use def expand(self) to reinitalize state
            else:
                total_count += 1
                pruned_count, total_count = cls.prune(child, min_visits, pruned_count, total_count)
        return pruned_count, total_count


    @classmethod
    def filename(cls):
        return f"data/{cls.__name__}_base64.py"

    @classmethod
    def load(cls):
        filename = cls.filename()
        loaded   = base64_file_load(filename)
        if loaded is not None:
            cls.save_node = loaded
            return loaded
        else:
            return None


    @classmethod
    def save(cls) -> Union[str,None]:
        if cls.persist == True and cls.save_node is not None:
            save_node    = cls.save_node

            start_time   = time.perf_counter()
            pruned_count, total_count = cls.prune(save_node)  # This reduces a 47MB base64 file down to 5Mb
            print(f'{cls.__name__}.save() - pruned {pruned_count:.0f}/{total_count:.0f} nodes '
                  + f'leaving {total_count-pruned_count:.0f} in {time.perf_counter() - start_time:.2f}s - ')

            filename = cls.filename()
            filesize = base64_file_save(save_node, filename)
            return filename
        return None



    ### Constructors and Lookups

    def create_child( self, action: int ) -> 'MontyCarloNode':
        result = result_action(self.bitboard, action, self.player_id)
        child  = None  # self.find_mirror_child(result, depth=1)  # BUG: using mirror hashes causes get_best_action() to return invalid moves
        if child is None:
            child = self.__class__(
                bitboard      = result,
                player_id     = next_player_id(self.player_id),
                parent        = self,
                parent_action = action,
                exploration   = self.exploration,
                **self.kwargs
            )
        self.children[action] = child
        self.is_expanded      = self._is_expanded()
        if self.is_expanded:
            self.on_expanded()
        return child


    def find_child( self, bitboard: np.array, depth=2 ) -> Union['MontyCarloNode', None]:
        # assert 0 <= depth <= 2

        if depth >= 0:
            if np.all( self.bitboard == bitboard ):
                return self
        if depth >= 1:
            for child in self.children:
                if child is None: continue
                if np.all( child.bitboard == bitboard ):
                    return child
        if depth >= 2:
            # Avoid recursion to prevent duplicate calls to hash_bitboard()
            for child in self.children:
                if child is None: continue
                for grandchild in child.children:
                    if grandchild is None: continue
                    if np.all( grandchild.bitboard == bitboard ):
                        return grandchild
        return None

    # # BUG: using mirror hashes causes get_best_action() to return invalid moves
    # def find_mirror_child( self, bitboard: np.array, depth=2 ) -> Union['MontyCarloNode', None]:
    #     # assert 0 <= depth <= 2
    #     mirror_hash = hash_bitboard(bitboard)
    #
    #     if depth >= 0:
    #         if self.mirror_hash == mirror_hash:
    #             return self
    #     if depth >= 1:
    #         for child in self.children:
    #             if child is None: continue
    #             if child.mirror_hash == mirror_hash:
    #                 return child
    #     if depth >= 2:
    #         # Avoid recursion to prevent duplicate calls to hash_bitboard()
    #         for child in self.children:
    #             if child is None: continue
    #             for grandchild in child.children:
    #                 if grandchild is None: continue
    #                 if grandchild.mirror_hash == mirror_hash:
    #                     return grandchild
    #     return None



    ### Properties

    def _is_expanded(self) -> bool:
        is_expanded = True
        for action in self.legal_moves:
            if self.children[action] is None:
                is_expanded = False
                break
        return is_expanded


    def get_unexpanded(self) -> List[int]:
        return [
            action
            for action in self.legal_moves
            if self.children[action] is None
        ]


    ### Action Selection

    def get_best_action(self) -> int:
        scores = [
            self.children[action].total_score
            if self.children[action] is not None else 0
            for action in self.legal_moves
        ]
        index  = np.argmax(scores)
        action = self.legal_moves[index]
        return action


    def get_exploration_action(self) -> int:
        scores = [
            self.children[action].ucb1_score
            if self.children[action] is not None else 0
            for action in self.legal_moves
        ]
        index  = np.argmax(scores)
        action = self.legal_moves[index]
        return action



    ### Scores

    def get_ucb1_score(self, node: 'MontyCarloNode') -> float:
        if node is None or node.total_visits == 0:
            return np.inf
        else:
            score = node.total_score / node.total_visits
            if node.parent is not None and node.parent.total_visits > 0:
                score += (
                    node.exploration * np.sqrt(2)
                    * np.log(node.parent.total_visits) / node.total_visits
                )
            return score


    @staticmethod
    def opponents_score(score: float):
        # assert 0 <= score <= 1
        return 1 - score



    ### Training and Backpropagation

    def single_run(self):
        if self.is_gameover:
            self.backpropagate(self.utility)
        elif not self.is_expanded:
            child = self.expand()
            score = child.simulate()    # score from the perspective of the other player
            child.backpropagate(score)
        else:
            # Recurse down tree, until a gameover or not expanded node is found
            action = self.get_exploration_action()
            child  = self.children[action]
            child.single_run()


    def expand(self) -> 'MontyCarloNode':
        # assert not self.is_gameover
        # assert not self.is_expanded

        unexpanded = self.get_unexpanded()
        # assert len(unexpanded)

        action     = np.random.choice(unexpanded)
        child      = self.create_child(action)
        return child

    def on_expanded(self) -> None:
        # Callback placeholder for any subclass hooks
        pass

    def simulate(self) -> float:
        return run_random_simulation(self.bitboard, self.player_id)


    def backpropagate(self, score: float):
        # child.simulate()  returns score for the player 2
        # child.total_score is accessed via the parent node, so score on this node is from the perspective of player 1
        node = self
        while node is not None:
            score = self.opponents_score(score)
            node.total_score  += score
            node.total_visits += 1
            node = node.parent      # when we reach the root: node.parent == None which terminates

        # get_ucb1_score() gets called 4x less often if we cache the value after backpropagation
        # get_ucb1_score() depends on parent.total_visits, so needs to be called after updating scores
        node = self
        while node is not None:
            node.ucb1_score = node.get_ucb1_score(node)
            node = node.parent      # when we reach the root: node.parent == None which terminates



    ### Agent
    @classmethod
    def agent(cls, **kwargs) -> Callable[[Struct, Struct],int]:
        def kaggle_agent( observation: Struct, _configuration_: Struct ):
            first_move_time = 0
            safety_time     = kwargs.get('safety_time', 0.25)
            start_time      = time.perf_counter()
            # configuration   = cast_configuration(_configuration_)

            player_id     = int(observation.mark)
            listboard     = np.array(observation.board, dtype=np.int8)
            bitboard      = list_to_bitboard(listboard)
            move_number   = get_move_number(bitboard)
            is_first_move = int(move_number < 2)
          # endtime       = start_time + _configuration_.timeout - safety_time - (first_move_time * is_first_move)
            endtime       = start_time +           1.25          - safety_time - (first_move_time * is_first_move)
            cls.init_class()  # ensure unique instance of cls.save_node between subclasses
            if cls.persist == True and cls.save_node is None:
                atexit.register(cls.save)
                cls.save_node = cls.load() or cls(empty_bitboard(), player_id=1)
                cls.root_nodes[1] = cls.root_nodes[2] = cls.save_node  # implement shared state

            root_node = cls.root_nodes[player_id]
            if root_node is None or root_node.find_child(bitboard, depth=2) is None:
                root_node = cls.root_nodes[player_id] = cls(
                    bitboard      = bitboard,
                    player_id     = player_id,
                    parent        = None,
                    # configuration = configuration,
                    **kwargs
                )
            else:
                root_node = cls.root_nodes[player_id] = cls.root_nodes[player_id].find_child(bitboard)
            # assert root_node is not None

            count = 0
            while time.perf_counter() < endtime:
                count += 1
                root_node.single_run()

            action     = root_node.get_best_action()
            time_taken = time.perf_counter() - start_time
            print(f'{cls.__name__}: p{player_id} action = {action} after {count} simulations in {time_taken:.3f}s')
            return int(action)

        kaggle_agent.__name__ = cls.__name__
        return kaggle_agent

def MontyCarloPure(**kwargs):
    # observation   = {'mark': 1, 'board': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}
    # configuration = {'columns': 7, 'rows': 6, 'inarow': 4, 'steps': 1000, 'timeout': 8}
    def MontyCarloPure(observation: Struct, configuration: Struct) -> int:
        return MontyCarloNode.agent(**kwargs)(observation, configuration)
    return MontyCarloPure

def MontyCarloPureKaggle(observation, configuration):
    return MontyCarloPure()(observation, configuration)


#####
##### END   agents/MontyCarlo/MontyCarloPure.py
#####

#####
##### START heuristics/BitboardGameoversHeuristic.py
#####

import math
import sys

# from core.ConnectXBBNN import *
# Hyperparameters
# from util.sigmoid import scaled_sigmoid

single_square_score = 0.1  # Mostly ignore single squares, that can make lines in 8 directions
double_attack_score = 0.5  # 0.5 == 100% winrate vs AlphaBetaAgent

min_score =  math.inf  # min_score: -32.3
max_score = -math.inf  # max_score:  26.4


# Profiler (Macbook Pro 2011): This vectorized implementation is actually slower than unvectorized
# bitboard_gameovers_heuristic	                call_count=9469	    time=1558	own_time=1141
# bitboard_gameovers_heuristic_unvectorized	    call_count=9469	    time=1879	own_time=1862  ( 20% slower)
# bitboard_gameovers_heuristic_slow	            call_count=9469	    time=3419	own_time=1893  (220% slower)
def bitboard_gameovers_heuristic( bitboard: np.ndarray, player_id: int ) -> float:
    """ For all possible connect4 gameover positions,
        check if a player has at least one token in position and that the opponent is not blocking
        return difference in score

        Winrates:
         55% vs AlphaBetaAgent - original heuristic
         70% vs AlphaBetaAgent - + np.log2(p1_can_play) % 1 == 0
         60% vs AlphaBetaAgent - + double_attack_score=1   without np.log2() (mostly draws)
         80% vs AlphaBetaAgent - + double_attack_score=1   with np.log2() (mostly wins)
         80% vs AlphaBetaAgent - + double_attack_score=2   with np.log2() (mostly wins)
         70% vs AlphaBetaAgent - + double_attack_score=4   with np.log2() (mostly wins)
         80% vs AlphaBetaAgent - + double_attack_score=8   with np.log2() (mostly wins)
        100% vs AlphaBetaAgent - + double_attack_score=0.5 with np.log2() (mostly wins)
    """

    p1_score = 0.0
    p2_score = 0.0

    invert_mask = sys.maxsize
    p1_tokens   = bitboard[0] & (bitboard[1] ^ invert_mask)
    p2_tokens   = bitboard[0] & (bitboard[1])

    for n in range(1):  # allow short circuit break statement
        p1_bitmasks = p1_tokens & gameovers
        p2_bitmasks = p2_tokens & gameovers
        p1_wins     = p1_bitmasks == gameovers
        p2_wins     = p2_bitmasks == gameovers

        # If we have 4 in a row, then immediately return infinity
        if np.any(p1_wins):
            p1_score = np.inf
            break
        if np.any(p2_wins):
            p2_score = np.inf
            break

        # Exclude any gameovers that contain moves from both players
        # np.log2() % 1 == 0 will be true for any bitmask containing only a single bit
        # p1_lines is a shorted array only containing matching gameover masks
        overlaps            = (p1_bitmasks != 0) & (p2_bitmasks != 0)
        p1_is_valid         = (p1_bitmasks != 0) & ~overlaps
        p2_is_valid         = (p2_bitmasks != 0) & ~overlaps
        p1_is_single_square = p1_is_valid & ( np.log2(p1_bitmasks, where=p1_is_valid ) % 1 == 0 )
        p2_is_single_square = p2_is_valid & ( np.log2(p2_bitmasks, where=p2_is_valid ) % 1 == 0 )

        p1_lines            = gameovers[ p1_is_valid & ~p1_is_single_square ]
        p2_lines            = gameovers[ p2_is_valid & ~p2_is_single_square ]

        # Offer a reduced score for any bitmask containing only a single bit
        p1_score += p1_lines.size + np.count_nonzero(p1_is_single_square) * single_square_score
        p2_score += p2_lines.size + np.count_nonzero(p2_is_single_square) * single_square_score

        # NOTE: Turns out that trying to be clever and creating a square matrix using np.roll() is actually slower!
        if p1_lines.size >= 2:
            for n1 in range(p1_lines.size):
                for n2 in range(n1+1, p1_lines.size):
                    gameover1 = p1_lines[n1]
                    gameover2 = p1_lines[n2]
                    overlap   = gameover1 & gameover2
                    if overlap == 0:              continue  # Ignore no overlap
                    if np.log2(overlap) % 1 != 0: continue  # Only count double_attacks with a single overlap square
                    p1_score += double_attack_score
        if p2_lines.size >= 2:
            for n1 in range(p2_lines.size):
                for n2 in range(n1+1, p2_lines.size):
                    gameover1 = p2_lines[n1]
                    gameover2 = p2_lines[n2]
                    overlap   = gameover1 & gameover2
                    if overlap == 0:              continue  # Ignore no overlap
                    if np.log2(overlap) % 1 != 0: continue  # Only count double_attacks with a single overlap square
                    p2_score += double_attack_score

    score = (p1_score - p2_score) if player_id == 1 else (p2_score - p1_score)
    # # assert np.math.isclose( score, bitboard_gameovers_heuristic_unvectorized(bitboard, player_id, gameovers), abs_tol=0.01), f'{score} != {bitboard_gameovers_heuristic_unvectorized(bitboard, player_id, gameovers)}'

    # global min_score, max_score
    # if score < min_score: min_score = score; print(f'min_score: {min_score}')  # min_score: -32.3
    # if score > max_score: max_score = score; print(f'max_score: {max_score}')  # max_score:  26.4
    return score


def bitboard_gameovers_heuristic_sigmoid(sigmoid_width = 6.0, sigmoid_height = 1.0):
    def _bitboard_gameovers_heuristic_sigmoid( bitboard: np.ndarray, player_id: int ) -> float:
        score = bitboard_gameovers_heuristic(bitboard, player_id)
        score = scaled_sigmoid(score, sigmoid_width, sigmoid_height)
        return score
    return _bitboard_gameovers_heuristic_sigmoid


### Previous implementations of the above code

# @njit
def bitboard_gameovers_heuristic_unvectorized( bitboard: np.ndarray, player_id: int, gameovers: np.ndarray = get_gameovers() ) -> float:
    invert_mask = sys.maxsize
    p1_tokens   = bitboard[0] & (bitboard[1] ^ invert_mask)
    p2_tokens   = bitboard[0] & (bitboard[1])

    p1_score = 0.0
    p2_score = 0.0
    p1_gameovers = []
    p2_gameovers = []
    for gameover in gameovers:
        p1_can_play = p1_tokens & gameover
        p2_can_play = p2_tokens & gameover
        if p1_can_play and not p2_can_play:
            if   p1_can_play == gameover:       p1_score += np.inf                # Connect 4
            elif np.log2(p1_can_play) % 1 == 0: p1_score += single_square_score   # Mostly ignore single square lines
            else:                               p1_score += 1; p1_gameovers.append(gameover);
        elif p2_can_play and not p1_can_play:
            if   p2_can_play == gameover:       p2_score += np.inf                # Connect 4
            elif np.log2(p2_can_play) % 1 == 0: p2_score += single_square_score   # Mostly ignore single square lines
            else:                               p2_score += 1; p2_gameovers.append(gameover);


    for n1 in range(len(p1_gameovers)):
        for n2 in range(n1+1, len(p1_gameovers)):
            gameover1 = p1_gameovers[n1]
            gameover2 = p1_gameovers[n2]
            overlap = gameover1 & gameover2
            if gameover1 == gameover2:    continue  # Ignore self
            if overlap == 0:              continue  # Ignore no overlap
            if np.log2(overlap) % 1 != 0: continue  # Only count double_attacks with a single overlap square
            p1_score += double_attack_score
    for n1 in range(len(p2_gameovers)):
        for n2 in range(n1+1, len(p2_gameovers)):
            gameover1 = p2_gameovers[n1]
            gameover2 = p2_gameovers[n2]
            overlap = gameover1 & gameover2
            if gameover1 == gameover2:    continue  # Ignore self
            if overlap == 0:              continue  # Ignore no overlap
            if np.log2(overlap) % 1 != 0: continue  # Only count double_attacks with a single overlap square
            p2_score += double_attack_score

    if player_id == 1:
        return p1_score - p2_score
    else:
        return p2_score - p1_score



# Profiler (Macbook Pro 2011): This vectorized implementation is actually slower than unvectorized
# bitboard_gameovers_heuristic	                call_count=9469	    time=1558	own_time=1141
# bitboard_gameovers_heuristic_unvectorized	    call_count=9469	    time=1879	own_time=1862  ( 20% slower)
# bitboard_gameovers_heuristic_slow	            call_count=9469	    time=3419	own_time=1893  (220% slower)
def bitboard_gameovers_heuristic_slow( bitboard: np.ndarray, player_id: int, gameovers: np.ndarray = get_gameovers() ) -> float:

    # Hyperparameters
    single_square_score = 0.1  # Mostly ignore single squares, that can make lines in 8 directions
    double_attack_score = 0.5  # 0.5 == 100% winrate vs AlphaBetaAgent

    scores = np.array([0.0, 0.0], dtype=np.float32)

    invert_mask = sys.maxsize
    tokens = np.array([
        bitboard[0] & (bitboard[1] ^ invert_mask),
        bitboard[0] & (bitboard[1]),
    ])
    for _ in range(1):  # allow short circuit break statement
        bitmasks = np.array([ token & gameovers for token in tokens ])
        wins     = bitmasks == gameovers
        if np.any(wins):
            if np.any(wins[0]): scores[0] = np.inf
            if np.any(wins[1]): scores[1] = np.inf
            break

        is_overlap = (bitmasks[0] != 0) & (bitmasks[1] != 0)
        is_valid   = (bitmasks    != 0) & ~is_overlap
        if not np.any(is_valid):
            break

        is_single_square = is_valid & ( np.log2(bitmasks, where=is_valid ) % 1 == 0 )
        is_multi_line    = is_valid & ~is_single_square
        scores += np.count_nonzero(is_multi_line, axis=-1) + np.count_nonzero(is_single_square, axis=-1) * single_square_score
        pass

        lines = [
            gameovers[ is_multi_line[0] ],
            gameovers[ is_multi_line[1] ]
        ]
        for player in range(len(lines)):
            player_lines = lines[player]
            if len(player_lines) <= 1: continue
            for n in range(player_lines.size):
                gameover = player_lines[n]
                overlaps = gameover & player_lines[n+1:]
                overlaps = overlaps[ (overlaps != 0) & (overlaps != gameover) ]  # Ignore empty and self
                double_attacks = np.count_nonzero(np.log2(overlaps) % 1 == 0)    # Only count double_attacks with a single overlap square
                scores[player] += double_attacks * double_attack_score

    score = (scores[0] - scores[1]) if player_id == 1 else (scores[1] - scores[0])
    # # assert np.math.isclose( score, bitboard_gameovers_heuristic_unvectorized(bitboard, player_id, gameovers), abs_tol=0.01), f'{score} != {bitboard_gameovers_heuristic_unvectorized(bitboard, player_id, gameovers)}'
    return score



#####
##### END   heuristics/BitboardGameoversHeuristic.py
#####

#####
##### START agents/MontyCarlo/MontyCarloHeuristic.py
#####

# This is a LinkedList implementation of MontyCarlo Tree Search
# but using bitboard_gameovers_heuristic() instead of run_random_simulation()
# Inspired by https://www.kaggle.com/matant/monte-carlo-tree-search-connectx
from struct import Struct
from typing import Callable
from typing import Dict

# from agents.MontyCarlo.MontyCarloPure import MontyCarloNode
# from core.ConnectXBBNN import *
# from heuristics.BitboardGameoversHeuristic import bitboard_gameovers_heuristic_sigmoid

Hyperparameters = namedtuple('hyperparameters', [])

class MontyCarloHeuristicNode(MontyCarloNode):
    heuristic_fn   =  bitboard_gameovers_heuristic_sigmoid
    heuristic_args = {}

    def __init__(
            self,
            bitboard:      np.ndarray,
            player_id:     int,
            parent:        Union['MontyCarloNode', None] = None,
            parent_action: Union[int,None]              = None,
            exploration:   float = 1.0,
            heuristic_fn:  Callable = None,
            heuristic_args: Dict    = None,
            **kwargs
    ):
        super().__init__(
            bitboard        = bitboard,
            player_id       = player_id,
            parent          = parent,
            parent_action   = parent_action,
            exploration     = exploration,
            heuristic_fn    = heuristic_fn,
            heuristic_args  = heuristic_args,
            **kwargs
        )
        # self.kwargs[] needs to be defined in order to pass arguments down to child nodes
        self.kwargs['heuristic_fn']   = self.heuristic_fn   = heuristic_fn   or self.__class__.heuristic_fn
        self.kwargs['heuristic_args'] = self.heuristic_args = heuristic_args or self.__class__.heuristic_args

    def simulate(self) -> float:
        self.heuristic = getattr(self, 'heuristic', self.heuristic_fn(**(self.heuristic_args or {})))
        score = self.heuristic(self.bitboard, self.player_id)
        return score

    # BUGFIX: pickle() throws exceptions for functions as local properties
    @classmethod
    def prune(cls, node: 'MontyCarloNode', *args, **kwargs):
        node.heuristic = None  # remove from pickle, will be restored by simulate()
        return super().prune(node, *args, **kwargs)


def MontyCarloHeuristic(**kwargs):
    # observation   = {'mark': 1, 'board': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}
    # configuration = {'columns': 7, 'rows': 6, 'inarow': 4, 'steps': 1000, 'timeout': 8}
    def MontyCarloHeuristic(observation: Struct, configuration: Struct) -> int:
        return MontyCarloHeuristicNode.agent(**kwargs)(observation, configuration)
    return MontyCarloHeuristic

def MontyCarloHeuristicKaggle(observation, configuration):
    return MontyCarloHeuristic()(observation, configuration)


#####
##### END   agents/MontyCarlo/MontyCarloHeuristic.py
#####

#####
##### START heuristics/BitsquaresHeuristic.py
#####

# from core.ConnectXBBNN import *


# Scores vs bitboard_gameovers_heuristic():
#   62% winrate @ reward_power=1
#   78% winrate @ reward_power=1.25
#   81% winrate @ reward_power=1.5
#   94% winrate @ reward_power=1.75
#   48% winrate @ reward_power=2
#   64% winrate @ reward_power=2.5
#   69% winrate @ reward_power=3
#   25% winrate @ reward_power=4
# from util.sigmoid import scaled_sigmoid


def bitsquares_heuristic(reward_power=1.75):
    def _bitsquares_heuristic(bitboard: np.ndarray, player_id: int, playable_lines = None):
        playable_lines = playable_lines or get_playable_lines_by_length(bitboard)
        scores = [ 0, 0 ]
        for player in [0,1]:
            for n in range(1, configuration.inarow+1):
                scores[player] += len(playable_lines[player][n]) * (n ** reward_power)

        score = (scores[0] - scores[1]) if player_id == 1 else (scores[1] - scores[0])
        return score
    return _bitsquares_heuristic


# Scores for: *sigmoid_width=1.75**: 1-in-a-row = 1 | 2-in-a-row = 3.4 | 3-in-a-row = 6.8
def bitsquares_heuristic_sigmoid(reward_power=1.75, sigmoid_width=7.0, sigmoid_height=1.0):
    heuristic = bitsquares_heuristic(reward_power=reward_power)
    def _bitsquares_heuristic_sigmoid(bitboard: np.ndarray, player_id: int, playable_lines = None) -> float:
        score = heuristic(bitboard=bitboard, player_id=player_id, playable_lines=playable_lines)
        score = scaled_sigmoid(score, sigmoid_width, sigmoid_height)
        return score
    return _bitsquares_heuristic_sigmoid


### Utility Functions

def get_playable_lines(bitboard: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    played_squares = bitboard[0]
    empty_squares  = mask_board  & ~bitboard[0]
    p1_tokens      = bitboard[0] & ~bitboard[1]
    p2_tokens      = bitboard[0] &  bitboard[1]

    # find gameovers masks that could be filled with empty squares
    p1_playable_lines = gameovers[:] & (p1_tokens & ~p2_tokens | empty_squares)
    p2_playable_lines = gameovers[:] & (p2_tokens & ~p1_tokens | empty_squares)
    p1_is_valid = (p1_playable_lines[:] == gameovers[:]) & (p1_playable_lines[:] & played_squares != 0)
    p2_is_valid = (p2_playable_lines[:] == gameovers[:]) & (p2_playable_lines[:] & played_squares != 0)

    p1_playable_lines = p1_playable_lines[p1_is_valid]
    p2_playable_lines = p2_playable_lines[p2_is_valid]
    return p1_playable_lines, p2_playable_lines


def get_playable_lines_by_length(bitboard: np.ndarray) -> List[List[np.ndarray]]:
    """
    Returns outputs[player][length] = gameovers[bitcount == length]
    """
    outputs        = [ [], [] ]
    player_tokens  = [ bitboard[0] & ~bitboard[1], bitboard[0] &  bitboard[1] ]
    playable_lines = get_playable_lines(bitboard)
    played_bits    = [ player_tokens[0] & playable_lines[0], player_tokens[1] & playable_lines[1] ]
    inarow         = configuration.inarow

    for player in [0,1]:
        is_gameover  = (playable_lines[player][:] == played_bits[player][:])
        is_singlebit = np.log2(played_bits[player][:]) % 1 == 0
        is_multibit  = ~is_gameover[:] & ~is_singlebit[:]
        bitcounts    = np.array([
            np.count_nonzero(
                bitcount_mask[:] & playable_lines[player][n] & player_tokens[player]
            ) if is_multibit[n]
            # else 1      if is_singlebit[n]
            # else inarow if is_gameover[n]
            else 0
            for n in range(len(playable_lines[player]))
        ])

        outputs[player]         = [ np.array([], dtype=np.int64) for _ in range(inarow + 1) ]
        outputs[player][1]      = playable_lines[player][ is_singlebit ]
        outputs[player][inarow] = playable_lines[player][ is_gameover  ]
        for n in range(2, inarow):
            outputs[player][n]  = playable_lines[player][ bitcounts == n ]
    return outputs

#####
##### END   heuristics/BitsquaresHeuristic.py
#####

#####
##### START agents/MontyCarlo/MontyCarloBitsquares.py
#####

# This is a LinkedList implementation of MontyCarlo Tree Search
# but using bitboard_gameovers_heuristic() instead of run_random_simulation()
# Inspired by https://www.kaggle.com/matant/monte-carlo-tree-search-connectx
from struct import Struct

# from agents.MontyCarlo.MontyCarloHeuristic import MontyCarloHeuristicNode
# from core.ConnectXBBNN import *
# from heuristics.BitsquaresHeuristic import bitsquares_heuristic_sigmoid

Hyperparameters = namedtuple('hyperparameters', [])

class MontyCarloBitsquaresNode(MontyCarloHeuristicNode):
    heuristic_fn   = bitsquares_heuristic_sigmoid
    heuristic_args = {}  # reward_power=1.75, sigmoid_width=7.0, sigmoid_height=1.0

class MontyCarloBitsquaresNode2(MontyCarloBitsquaresNode):
    pass


def MontyCarloBitsquares(**kwargs):
    # observation   = {'mark': 1, 'board': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}
    # configuration = {'columns': 7, 'rows': 6, 'inarow': 4, 'steps': 1000, 'timeout': 8}
    def MontyCarloBitsquares(observation: Struct, configuration: Struct) -> int:
        return MontyCarloBitsquaresNode.agent(**kwargs)(observation, configuration)
    return MontyCarloBitsquares

def MontyCarlo_BtS_JMcGuigan(observation, configuration):
    return MontyCarloBitsquares()(observation, configuration)


#####
##### END   agents/MontyCarlo/MontyCarloBitsquares.py
#####

##### 
##### /ai-games/kaggle_compile.py agents/MontyCarlo/MontyCarloBitsquares.py
##### 
##### 2024-01-28 08:30:57+00:00
##### 
##### origin	https://github.com/JamesMcGuigan/ai-games.git (fetch)
##### origin	https://github.com/JamesMcGuigan/ai-games.git (push)
##### 
##### * master eac436d Revert: Rock Paper Scissors | Random Seed Search | tweak unit tests
##### 
##### eac436d23e03624c2245838138e9608cb13d2f1f
##### 

Writing MCTS_Bitboard_Bitsquares_Heuristic_JamesMcGuigan.py


In [12]:
%run MCTS_Bitboard_Bitsquares_Heuristic_JamesMcGuigan.py

## [The Fast mini Max **II**](https://www.kaggle.com/code/martynovandrey/the-fast-mini-max-ii) - [*Martynov Andrey*](https://www.kaggle.com/martynovandrey)

In [13]:
%%writefile agent_The_Fast_mini_Max_2.py

def agent_TheFast_mini_Max_2(obs, config):
    import numpy as np
    import random
    import time

    # Precalculate some constants for better performance
    W = config.inarow
    R = config.rows
    C = config.columns
    Rp1 = R + 1
    Rm1 = R - 1
    RmW = R - W
    RmWp1 = R - W + 1
    Cp1 = C + 1
    Cm1 = C - 1
    Rp1Cp1 = Rp1 * Cp1
    Ct2 = C * 2
    Cp1t2 = Cp1 * 2
    Cm1t2 = Cm1 * 2
    Wmask = 2**W - 1
    DELAY = 0.5
   
    # the order of calculation the scores of a move, nearest to the center first, 
    # in the case of 7 columns it's [3, 2, 4, 1, 5, 0, 6]
    ORDER = [C//2 - i//2 - 1 if i%2 else C//2 + i//2 for i in range(C)]
    
    # Some utility functions
    
    def step_number(grid):
        # returns step of the game
        empty = 0
        for i in range(R):
            for j in range(C):
                if grid[i][j] == 0:
                    empty += 1
        return R*C - empty

    def is_first_player(step):
        return step%2 == 0
    
    def int_to_bin(x):
        # 64-bit integer to 64-letter string
        return bin(x)[2:].zfill(64)
    
    
    class State:
        '''
        Store a board as dict of two integers
        pos[1] - position of ones
        pos[2] - position of twos
        '''
        
        def __init__(self):
            self.pos = dict()

        def valid_moves(self):
            mask = self.pos[1] | self.pos[2]
            return [i for i in ORDER if not (mask >> Rm1+C*i) & 1]         

        def from_grid(grid):
            # converts 2-D array of 0, 1, 2 ("grid") to state
            state = State()
            matrix = np.array([0 for i in range(Rp1Cp1)]).reshape(Rp1, Cp1)
            for r in range(R):
                for c in range(C):
                    matrix[r+1, c] = grid[r, c]
            position1, position2 = '', ''
            for c in range(C, -1, -1):
                for r in range(0, Rp1):
                    position1 += ['0', '1'][matrix[r,c] == 1]
                    position2 += ['0', '1'][matrix[r,c] == 2]
            state.pos[1] = int(position1, 2)
            state.pos[2] = int(position2, 2)
            return state

        def to_grid(self):
            position1 = int_to_bin(self.pos[1])
            position2 = int_to_bin(self.pos[2])
            matrix = np.array([0 for i in range(Rp1Cp1)]).reshape(Rp1, Cp1)
            for c in range(0, Cp1):
                for r in range(R, -1, -1):
                    if position1[-1] == '1':
                        matrix[r,c] = 1
                    position1 = position1[:-1]
                    if position2[-1] == '1':
                        matrix[r,c] = 2
                    position2 = position2[:-1]
            return matrix[1:, :-1]

        def connected_four(self, mark):
            # checks if the state has for in a row of mark
            position = self.pos[mark]
            # Horizontal check
            m = position & (position >> C)
            if m & (m >> Ct2): return True
            # Diagonal \
            m = position & (position >> Cm1)
            if m & (m >> Cm1t2): return True
            # Diagonal /
            m = position & (position >> Cp1)
            if m & (m >> Cp1t2): return True
            # Vertical
            m = position & (position >> 1)
            if m & (m >> 2): return True
            # Nothing found
            return False

        def make_move(self, col, mark):
            # returns state after playing piece mark in column col
            mask = self.pos[1] | self.pos[2]
            new_state = State()
            new_mask = mask | (mask + (1 << (col*C)))
            new_state.pos[mark] = self.pos[3-mark] ^ new_mask
            new_state.pos[3-mark] = self.pos[3-mark]
            return new_state

        def doublewin(self, mark):
            # helper for heuristic, checks if player has
            # two winning moves simultaneously or two in a row in the 
            # same column
            win = 0
            for col in self.valid_moves():
                next_state = self.make_move(col, mark)
                if next_state.connected_four(mark):
                    next_state_opp =  self.make_move(col, 3-mark)
                    if next_state_opp.make_move(col, mark).connected_four(mark):
                        return True # win with this column first or second move, unstoppable
                    win += 1
                    if win == 2:
                        return True # more than one winning move
            return False 
        

        def num23(self, mark):
            MASKS = [{4: 60, 3: [28, 44, 52, 56], 2: [12, 20, 24, 36, 40, 48]}, {4: 30, 3: [14, 22, 26, 28], 2: [6, 10, 12, 18, 20, 24]}, {4: 15, 3: [7, 11, 13, 14], 2: [3, 5, 6, 9, 10, 12]}, {4: 7680, 3: [3584, 5632, 6656, 7168], 2: [1536, 2560, 3072, 4608, 5120, 6144]}, {4: 3840, 3: [1792, 2816, 3328, 3584], 2: [768, 1280, 1536, 2304, 2560, 3072]}, {4: 1920, 3: [896, 1408, 1664, 1792], 2: [384, 640, 768, 1152, 1280, 1536]}, {4: 983040, 3: [458752, 720896, 851968, 917504], 2: [196608, 327680, 393216, 589824, 655360, 786432]}, {4: 491520, 3: [229376, 360448, 425984, 458752], 2: [98304, 163840, 196608, 294912, 327680, 393216]}, {4: 245760, 3: [114688, 180224, 212992, 229376], 2: [49152, 81920, 98304, 147456, 163840, 196608]}, {4: 125829120, 3: [58720256, 92274688, 109051904, 117440512], 2: [25165824, 41943040, 50331648, 75497472, 83886080, 100663296]}, {4: 62914560, 3: [29360128, 46137344, 54525952, 58720256], 2: [12582912, 20971520, 25165824, 37748736, 41943040, 50331648]}, {4: 31457280, 3: [14680064, 23068672, 27262976, 29360128], 2: [6291456, 10485760, 12582912, 18874368, 20971520, 25165824]}, {4: 16106127360, 3: [7516192768, 11811160064, 13958643712, 15032385536], 2: [3221225472, 5368709120, 6442450944, 9663676416, 10737418240, 12884901888]}, {4: 8053063680, 3: [3758096384, 5905580032, 6979321856, 7516192768], 2: [1610612736, 2684354560, 3221225472, 4831838208, 5368709120, 6442450944]}, {4: 4026531840, 3: [1879048192, 2952790016, 3489660928, 3758096384], 2: [805306368, 1342177280, 1610612736, 2415919104, 2684354560, 3221225472]}, {4: 2061584302080, 3: [962072674304, 1511828488192, 1786706395136, 1924145348608], 2: [412316860416, 687194767360, 824633720832, 1236950581248, 1374389534720, 1649267441664]}, {4: 1030792151040, 3: [481036337152, 755914244096, 893353197568, 962072674304], 2: [206158430208, 343597383680, 412316860416, 618475290624, 687194767360, 824633720832]}, {4: 515396075520, 3: [240518168576, 377957122048, 446676598784, 481036337152], 2: [103079215104, 171798691840, 206158430208, 309237645312, 343597383680, 412316860416]}, {4: 263882790666240, 3: [123145302310912, 193514046488576, 228698418577408, 246290604621824], 2: [52776558133248, 87960930222080, 105553116266496, 158329674399744, 175921860444160, 211106232532992]}, {4: 131941395333120, 3: [61572651155456, 96757023244288, 114349209288704, 123145302310912], 2: [26388279066624, 43980465111040, 52776558133248, 79164837199872, 87960930222080, 105553116266496]}, {4: 65970697666560, 3: [30786325577728, 48378511622144, 57174604644352, 61572651155456], 2: [13194139533312, 21990232555520, 26388279066624, 39582418599936, 43980465111040, 52776558133248]}, {4: 67637280, 3: [67637248, 67633184, 67112992, 528416], 2: [67633152, 67112960, 528384, 67108896, 524320, 4128]}, {4: 8657571840, 3: [8657567744, 8657047552, 8590462976, 67637248], 2: [8657043456, 8590458880, 67633152, 8589938688, 67112960, 528384]}, {4: 1108169195520, 3: [1108168671232, 1108102086656, 1099579260928, 8657567744], 2: [1108101562368, 1099578736640, 8657043456, 1099512152064, 8590458880, 67633152]}, {4: 141845657026560, 3: [141845589917696, 141837067091968, 140746145398784, 1108168671232], 2: [141836999983104, 140746078289920, 1108101562368, 140737555464192, 1099578736640, 8657043456]}, {4: 33818640, 3: [33818624, 33816592, 33556496, 264208], 2: [33816576, 33556480, 264192, 33554448, 262160, 2064]}, {4: 4328785920, 3: [4328783872, 4328523776, 4295231488, 33818624], 2: [4328521728, 4295229440, 33816576, 4294969344, 33556480, 264192]}, {4: 554084597760, 3: [554084335616, 554051043328, 549789630464, 4328783872], 2: [554050781184, 549789368320, 4328521728, 549756076032, 4295229440, 33816576]}, {4: 70922828513280, 3: [70922794958848, 70918533545984, 70373072699392, 554084335616], 2: [70918499991552, 70373039144960, 554050781184, 70368777732096, 549789368320, 4328521728]}, {4: 16909320, 3: [16909312, 16908296, 16778248, 132104], 2: [16908288, 16778240, 132096, 16777224, 131080, 1032]}, {4: 2164392960, 3: [2164391936, 2164261888, 2147615744, 16909312], 2: [2164260864, 2147614720, 16908288, 2147484672, 16778240, 132096]}, {4: 277042298880, 3: [277042167808, 277025521664, 274894815232, 2164391936], 2: [277025390592, 274894684160, 2164260864, 274878038016, 2147614720, 16908288]}, {4: 35461414256640, 3: [35461397479424, 35459266772992, 35186536349696, 277042167808], 2: [35459249995776, 35186519572480, 277025390592, 35184388866048, 274894684160, 2164260864]}, {4: 8454660, 3: [8454656, 8454148, 8389124, 66052], 2: [8454144, 8389120, 66048, 8388612, 65540, 516]}, {4: 1082196480, 3: [1082195968, 1082130944, 1073807872, 8454656], 2: [1082130432, 1073807360, 8454144, 1073742336, 8389120, 66048]}, {4: 138521149440, 3: [138521083904, 138512760832, 137447407616, 1082195968], 2: [138512695296, 137447342080, 1082130432, 137439019008, 1073807360, 8454144]}, {4: 17730707128320, 3: [17730698739712, 17729633386496, 17593268174848, 138521083904], 2: [17729624997888, 17593259786240, 138512695296, 17592194433024, 137447342080, 1082130432]}, {4: 4227330, 3: [4227328, 4227074, 4194562, 33026], 2: [4227072, 4194560, 33024, 4194306, 32770, 258]}, {4: 541098240, 3: [541097984, 541065472, 536903936, 4227328], 2: [541065216, 536903680, 4227072, 536871168, 4194560, 33024]}, {4: 69260574720, 3: [69260541952, 69256380416, 68723703808, 541097984], 2: [69256347648, 68723671040, 541065216, 68719509504, 536903680, 4227072]}, {4: 8865353564160, 3: [8865349369856, 8864816693248, 8796634087424, 69260541952], 2: [8864812498944, 8796629893120, 69256347648, 8796097216512, 68723671040, 541065216]}, {4: 2113665, 3: [2113664, 2113537, 2097281, 16513], 2: [2113536, 2097280, 16512, 2097153, 16385, 129]}, {4: 270549120, 3: [270548992, 270532736, 268451968, 2113664], 2: [270532608, 268451840, 2113536, 268435584, 2097280, 16512]}, {4: 34630287360, 3: [34630270976, 34628190208, 34361851904, 270548992], 2: [34628173824, 34361835520, 270532608, 34359754752, 268451840, 2113536]}, {4: 4432676782080, 3: [4432674684928, 4432408346624, 4398317043712, 34630270976], 2: [4432406249472, 4398314946560, 34628173824, 4398048608256, 34361835520, 270532608]}, {4: 8521760, 3: [8521728, 8519712, 8390688, 133152], 2: [8519680, 8390656, 133120, 8388640, 131104, 2080]}, {4: 1090785280, 3: [1090781184, 1090523136, 1074008064, 17043456], 2: [1090519040, 1074003968, 17039360, 1073745920, 16781312, 266240]}, {4: 139620515840, 3: [139619991552, 139586961408, 137473032192, 2181562368], 2: [139586437120, 137472507904, 2181038080, 137439477760, 2148007936, 34078720]}, {4: 17871426027520, 3: [17871358918656, 17867131060224, 17596548120576, 279239983104], 2: [17867063951360, 17596481011712, 279172874240, 17592253153280, 274945015808, 4362076160]}, {4: 4260880, 3: [4260864, 4259856, 4195344, 66576], 2: [4259840, 4195328, 66560, 4194320, 65552, 1040]}, {4: 545392640, 3: [545390592, 545261568, 537004032, 8521728], 2: [545259520, 537001984, 8519680, 536872960, 8390656, 133120]}, {4: 69810257920, 3: [69809995776, 69793480704, 68736516096, 1090781184], 2: [69793218560, 68736253952, 1090519040, 68719738880, 1074003968, 17039360]}, {4: 8935713013760, 3: [8935679459328, 8933565530112, 8798274060288, 139619991552], 2: [8933531975680, 8798240505856, 139586437120, 8796126576640, 137472507904, 2181038080]}, {4: 2130440, 3: [2130432, 2129928, 2097672, 33288], 2: [2129920, 2097664, 33280, 2097160, 32776, 520]}, {4: 272696320, 3: [272695296, 272630784, 268502016, 4260864], 2: [272629760, 268500992, 4259840, 268436480, 4195328, 66560]}, {4: 34905128960, 3: [34904997888, 34896740352, 34368258048, 545390592], 2: [34896609280, 34368126976, 545259520, 34359869440, 537001984, 8519680]}, {4: 4467856506880, 3: [4467839729664, 4466782765056, 4399137030144, 69809995776], 2: [4466765987840, 4399120252928, 69793218560, 4398063288320, 68736253952, 1090519040]}, {4: 16843009, 3: [16843008, 16842753, 16777473, 65793], 2: [16842752, 16777472, 65792, 16777217, 65537, 257]}, {4: 2155905152, 3: [2155905024, 2155872384, 2147516544, 8421504], 2: [2155872256, 2147516416, 8421376, 2147483776, 8388736, 32896]}, {4: 275955859456, 3: [275955843072, 275951665152, 274882117632, 1077952512], 2: [275951648768, 274882101248, 1077936128, 274877923328, 1073758208, 4210688]}, {4: 35322350010368, 3: [35322347913216, 35321813139456, 35184911056896, 137977921536], 2: [35321811042304, 35184908959744, 137975824384, 35184374185984, 137441050624, 538968064]}, {4: 33686018, 3: [33686016, 33685506, 33554946, 131586], 2: [33685504, 33554944, 131584, 33554434, 131074, 514]}, {4: 4311810304, 3: [4311810048, 4311744768, 4295033088, 16843008], 2: [4311744512, 4295032832, 16842752, 4294967552, 16777472, 65792]}, {4: 551911718912, 3: [551911686144, 551903330304, 549764235264, 2155905024], 2: [551903297536, 549764202496, 2155872256, 549755846656, 2147516416, 8421376]}, {4: 70644700020736, 3: [70644695826432, 70643626278912, 70369822113792, 275955843072], 2: [70643622084608, 70369817919488, 275951648768, 70368748371968, 274882101248, 1077936128]}, {4: 67372036, 3: [67372032, 67371012, 67109892, 263172], 2: [67371008, 67109888, 263168, 67108868, 262148, 1028]}, {4: 8623620608, 3: [8623620096, 8623489536, 8590066176, 33686016], 2: [8623489024, 8590065664, 33685504, 8589935104, 33554944, 131584]}, {4: 1103823437824, 3: [1103823372288, 1103806660608, 1099528470528, 4311810048], 2: [1103806595072, 1099528404992, 4311744512, 1099511693312, 4295032832, 16842752]}, {4: 141289400041472, 3: [141289391652864, 141287252557824, 140739644227584, 551911686144], 2: [141287244169216, 140739635838976, 551903297536, 140737496743936, 549764202496, 2155872256]}]
            n12, n13, n22, n23 = 0, 0, 0, 0
            s1, s2 = self.pos[mark], self.pos[3-mark]
            for m in MASKS:
                w1 = s1 & m[4]
                w2 = s2 & m[4]
                if not w2 and w1:
                    for m3 in m[3]:
                        if w1 == m3:
                            n13 += 1
                    for m2 in m[2]:
                        if w1 == m2:
                            n12 += 1
                if not w1 and w2:
                    for m3 in m[3]:
                        if w2 == m3:
                            n23 += 1
                    for m2 in m[2]:
                        if w2 == m2:
                            n22 += 1
            return  n12, n13, n22, n23

        def count_even_odd(self, mark):
            # helper for heuristic
            # calculate number of marks in even and odd rows
            odd, even = 0, 0
            for row in range(R):
                rd2 = row%2     
                for col in range(C):
                    if (self.pos[mark] >> (C*col+row)) & 1:
                        if rd2 == 0:
                            odd += 1
                        else:
                            even += 1
            return even, odd 

        def heuristic(self, mark, maximizingPlayer):
            # the heuristic uses numbers of 2-marks and 3 -marks windows
            # numbers of marks in the even and odd rows
            # and if player or opponent has "doublewin"

            num_twos, num_threes, num_twos_opp, num_threes_opp = self.num23(mark)
            score = 900000*num_threes - 900000*num_threes_opp + 30000*num_twos - 30000*num_twos_opp

            num_evens, num_odds = self.count_even_odd(mark)
            if first: 
                even_odd_rate = num_odds - num_evens
            else:
                even_odd_rate = num_evens - num_odds
            score += 100 * even_odd_rate

#             if num_threes > 1:
#                 if self.doublewin(mark) : score = 1e8
#             if num_threes_opp > 1:
#                 if self.doublewin(3-mark) : score = -1e8
            return score 

        def score_move(self, col, mark, nsteps):
            next_state = self.make_move(col, mark)
            return minimax(next_state, nsteps-1, -np.Inf, np.Inf, False, mark)
       
    # Minimax implementation
    def minimax(state, depth, a, b, maximizingPlayer, mark):
        
        if time.time() > FINISH: # timeout
            return 0

        if state.connected_four(mark): # player win
            return np.inf
        if state.connected_four(3-mark): # opponent win
            return -np.inf
        if state.pos[1] | state.pos[2] == 279258638311359: # fullboard mask, tie
            return 0

        if depth == 0: # leaf node
            key = (state.pos[1], state.pos[2]) # the key for store state in the dict
            if key in STATES: # heuristic for the state is in tne dict
                return STATES[key] # get it from the dict
            # new state, calculate heuristic
            h = state.heuristic(mark, maximizingPlayer=maximizingPlayer)
            STATES[key] = h # and store it in the dict
            return h
 
        # MiniMax with alpha-beta pruning
    
        valid_moves = state.valid_moves()
       
        if maximizingPlayer:
            value = -np.Inf
            for col in valid_moves:
                child = state.make_move(col, mark)
                value = max(value, minimax(child, depth-1, a, b, False, mark))
                if value >= b:
                    break
                a = max(a, value)
            return value
        else:
            value = np.Inf
            for col in valid_moves:
                child = state.make_move(col, 3-mark)
                value = min(value, minimax(child, depth-1, a, b, True, mark))
                if value <= a:
                    break
                b = min(b, value)
            return value 
    
    # The agent begins to work
    
    START = time.time()
    FINISH = START + DELAY
    
    # the dict to store heuristics, a la transposition table
    STATES = {}  
    
    # the depth to start with
    N_STEPS = 4
    
    # Convert the board to a 2D grid and then the grig  to state
    grid = np.asarray(obs.board).reshape(R, C)
    state = State.from_grid(grid)
    
    # the best move for the first move is "3", let's play it
    step = step_number(grid)
    if step < 2:
        return 3
    
    # remember if the agent is the first player, for heuristic
    first = is_first_player(step)

    valid_moves = state.valid_moves()
    # check if we have winning move
    for col in valid_moves:
        next_state = state.make_move(col, obs.mark)
        if next_state.connected_four(obs.mark):
            # yes, we have, let's play it
            return col
        
    # Use the heuristic to assign a score to each possible board in the next step
    # the agent is rather fast, so it starts with six steps to look ahead
    scores = dict(zip(valid_moves, [state.score_move(col, obs.mark, N_STEPS) for col in valid_moves]))

    # Repeat with one step more depth
    while  N_STEPS < 42 - step:
        N_STEPS += 1

        # no need to calculate scores for the moves we already know are the way to loss
        for vm in valid_moves:
            if scores[vm] == -np.inf:
                valid_moves.remove(vm)
        if len(valid_moves) == 0: # we loss anyway, surrender :(
            break
            
        scores_2 = {}
        for col in valid_moves:
            score = state.score_move(col, obs.mark, N_STEPS)
            if score == np.inf:
                # we have the move to win with, let's play it
                return col
            scores_2[col] = score

        if time.time() > FINISH or np.amax(list(scores_2.values())) <= -np.inf:
            # timeout, the search with the next depth isn't complete
            # leave the previous scores as is
            break
        # update the scores withe the result of the next depth search
        scores = scores_2.copy()
    
    # Get a list of columns (moves) that maximize the heuristic
    max_cols = [key for key in scores.keys() if scores[key] == max(scores.values())]

    # Select the best as nearest to the center of the board from the maximizing columns
    if len(max_cols) >0:
        if 3 in max_cols:
            return 3
        if 2 in max_cols and 4 in max_cols:
            return random.choice([2, 4])
        if 2 in max_cols:
            return 2
        if 4 in max_cols:
            return 4
        
        if 1 in max_cols and 5 in max_cols:
            return random.choice([1, 5])
        if 1 in max_cols:
            return 1
        if 5 in max_cols:
            return 5
        
        if 0 in max_cols and 6 in max_cols:
            return random.choice([0, 6])
        if 0 in max_cols:
            return 0
        if 6 in max_cols:
            return 6
        
        col = random.choice(max_cols)
        return col
    col =  random.choice(max_cols)
    return col 

Writing agent_The_Fast_mini_Max_2.py


In [14]:
%run agent_The_Fast_mini_Max_2.py

## [The Fast mini Max **IV**](https://www.kaggle.com/code/martynovandrey/the-fast-mini-max-iv) - [*Martynov Andrey*](https://www.kaggle.com/martynovandrey)

In [15]:
%%writefile agent_The_Fast_mini_Max_4.py

def agent_TheFast_mini_Max_4(obs, config):
    import numpy as np
    import random
    import time

    # Precalculate some constants for better performance
    W = config.inarow
    R = config.rows
    C = config.columns
    Rp1 = R + 1
    Rm1 = R - 1
    RmW = R - W
    RmWp1 = R - W + 1
    Cp1 = C + 1
    Cm1 = C - 1
    Rp1Cp1 = Rp1 * Cp1
    Ct2 = C * 2
    Cp1t2 = Cp1 * 2
    Cm1t2 = Cm1 * 2
    Wmask = 2**W - 1
    RC = R*C
    DELAY = 0.5
                
    # the order of calculation the scores of a move, nearest to the center first, 
    # in the case of 7 columns it's [3, 2, 4, 1, 5, 0, 6]
    ORDER = [C//2 - i//2 - 1 if i%2 else C//2 + i//2 for i in range(C)]
    
    # Some utility functions
    
    def step_number(grid):
        # returns step of the game
        empty = 0
        for i in range(R):
            for j in range(C):
                if grid[i][j] == 0:
                    empty += 1
        return R*C - empty

    def is_first_player(step):
        return step%2 == 0
    
    def int_to_bin(x):
        # 64-bit integer to 64-letter string
        return bin(x)[2:].zfill(64)
    
    def grid_to_matrix(grid):
        matrix = np.array([0 for i in range(Rp1Cp1)]).reshape(Rp1, Cp1)
        for r in range(R):
            for c in range(C):
                matrix[r+1, c] = grid[r, c]
        return matrix

    def grid_to_state(grid):
        matrix = grid_to_matrix(grid)
        position1, position2 = '', ''
        for c in range(C, -1, -1):
            for r in range(0, Rp1):
                if matrix[r,c] == 1:
                    position1 += '1'
                else:
                    position1 += '0'
                if matrix[r,c] == 2:
                    position2 += '1'
                else:
                    position2 += '0'
        return {1:int(position1, 2), 2:int(position2, 2)}

    def state_to_grid(state):
        position1 = state[1]
        position2 = state[2]
        matrix = np.array([[0, 0, 0, 0, 0, 0, 0, 0],
                           [0, 0, 0, 0, 0, 0, 0, 0],
                           [0, 0, 0, 0, 0, 0, 0, 0],
                           [0, 0, 0, 0, 0, 0, 0, 0],
                           [0, 0, 0, 0, 0, 0, 0, 0],
                           [0, 0, 0, 0, 0, 0, 0, 0],
                           [0, 0, 0, 0, 0, 0, 0, 0]])
        for c in range(0, Cp1):
            for r in range(R, -1, -1):
                if position1 & 1:
                    matrix[r,c] = 1
                position1 = position1 >> 1
                if position2 & 1:
                    matrix[r,c] = 2
                position2 = position2 >> 1
        return matrix[1:, :-1]

    def make_move(state, col, mark):
        mask = state[1] | state[2]
        new_state = state.copy()
        new_state[mark] = state[3-mark] ^ (mask | (mask + (1 << (col*7))))
        return new_state

    def connected_four(position):
        # Horizontal check
        m = position & (position >> 7)
        if m & (m >> 14): return True
        # Diagonal \
        m = position & (position >> 6)
        if m & (m >> 12): return True
        # Diagonal /
        m = position & (position >> 8)
        if m & (m >> 16): return True
        # Vertical
        m = position & (position >> 1)
        if m & (m >> 2): return True
        # Nothing found
        return False

    def int_to_bin(x):
        return bin(x)[2:].zfill(64)

    def valid_moves_list(state):
        m = state[1] | state[2]
        return [i for i in [3,2,4,1,5,0,6] if not (m >> 5+7*i) & 1]
   
    MASKS = [{4: 60, 3: [28, 44, 52, 56], 2: [12, 20, 24, 36, 40, 48]}, {4: 30, 3: [14, 22, 26, 28], 2: [6, 10, 12, 18, 20, 24]}, {4: 15, 3: [7, 11, 13, 14], 2: [3, 5, 6, 9, 10, 12]}, {4: 7680, 3: [3584, 5632, 6656, 7168], 2: [1536, 2560, 3072, 4608, 5120, 6144]}, {4: 3840, 3: [1792, 2816, 3328, 3584], 2: [768, 1280, 1536, 2304, 2560, 3072]}, {4: 1920, 3: [896, 1408, 1664, 1792], 2: [384, 640, 768, 1152, 1280, 1536]}, {4: 983040, 3: [458752, 720896, 851968, 917504], 2: [196608, 327680, 393216, 589824, 655360, 786432]}, {4: 491520, 3: [229376, 360448, 425984, 458752], 2: [98304, 163840, 196608, 294912, 327680, 393216]}, {4: 245760, 3: [114688, 180224, 212992, 229376], 2: [49152, 81920, 98304, 147456, 163840, 196608]}, {4: 125829120, 3: [58720256, 92274688, 109051904, 117440512], 2: [25165824, 41943040, 50331648, 75497472, 83886080, 100663296]}, {4: 62914560, 3: [29360128, 46137344, 54525952, 58720256], 2: [12582912, 20971520, 25165824, 37748736, 41943040, 50331648]}, {4: 31457280, 3: [14680064, 23068672, 27262976, 29360128], 2: [6291456, 10485760, 12582912, 18874368, 20971520, 25165824]}, {4: 16106127360, 3: [7516192768, 11811160064, 13958643712, 15032385536], 2: [3221225472, 5368709120, 6442450944, 9663676416, 10737418240, 12884901888]}, {4: 8053063680, 3: [3758096384, 5905580032, 6979321856, 7516192768], 2: [1610612736, 2684354560, 3221225472, 4831838208, 5368709120, 6442450944]}, {4: 4026531840, 3: [1879048192, 2952790016, 3489660928, 3758096384], 2: [805306368, 1342177280, 1610612736, 2415919104, 2684354560, 3221225472]}, {4: 2061584302080, 3: [962072674304, 1511828488192, 1786706395136, 1924145348608], 2: [412316860416, 687194767360, 824633720832, 1236950581248, 1374389534720, 1649267441664]}, {4: 1030792151040, 3: [481036337152, 755914244096, 893353197568, 962072674304], 2: [206158430208, 343597383680, 412316860416, 618475290624, 687194767360, 824633720832]}, {4: 515396075520, 3: [240518168576, 377957122048, 446676598784, 481036337152], 2: [103079215104, 171798691840, 206158430208, 309237645312, 343597383680, 412316860416]}, {4: 263882790666240, 3: [123145302310912, 193514046488576, 228698418577408, 246290604621824], 2: [52776558133248, 87960930222080, 105553116266496, 158329674399744, 175921860444160, 211106232532992]}, {4: 131941395333120, 3: [61572651155456, 96757023244288, 114349209288704, 123145302310912], 2: [26388279066624, 43980465111040, 52776558133248, 79164837199872, 87960930222080, 105553116266496]}, {4: 65970697666560, 3: [30786325577728, 48378511622144, 57174604644352, 61572651155456], 2: [13194139533312, 21990232555520, 26388279066624, 39582418599936, 43980465111040, 52776558133248]}, {4: 67637280, 3: [67637248, 67633184, 67112992, 528416], 2: [67633152, 67112960, 528384, 67108896, 524320, 4128]}, {4: 8657571840, 3: [8657567744, 8657047552, 8590462976, 67637248], 2: [8657043456, 8590458880, 67633152, 8589938688, 67112960, 528384]}, {4: 1108169195520, 3: [1108168671232, 1108102086656, 1099579260928, 8657567744], 2: [1108101562368, 1099578736640, 8657043456, 1099512152064, 8590458880, 67633152]}, {4: 141845657026560, 3: [141845589917696, 141837067091968, 140746145398784, 1108168671232], 2: [141836999983104, 140746078289920, 1108101562368, 140737555464192, 1099578736640, 8657043456]}, {4: 33818640, 3: [33818624, 33816592, 33556496, 264208], 2: [33816576, 33556480, 264192, 33554448, 262160, 2064]}, {4: 4328785920, 3: [4328783872, 4328523776, 4295231488, 33818624], 2: [4328521728, 4295229440, 33816576, 4294969344, 33556480, 264192]}, {4: 554084597760, 3: [554084335616, 554051043328, 549789630464, 4328783872], 2: [554050781184, 549789368320, 4328521728, 549756076032, 4295229440, 33816576]}, {4: 70922828513280, 3: [70922794958848, 70918533545984, 70373072699392, 554084335616], 2: [70918499991552, 70373039144960, 554050781184, 70368777732096, 549789368320, 4328521728]}, {4: 16909320, 3: [16909312, 16908296, 16778248, 132104], 2: [16908288, 16778240, 132096, 16777224, 131080, 1032]}, {4: 2164392960, 3: [2164391936, 2164261888, 2147615744, 16909312], 2: [2164260864, 2147614720, 16908288, 2147484672, 16778240, 132096]}, {4: 277042298880, 3: [277042167808, 277025521664, 274894815232, 2164391936], 2: [277025390592, 274894684160, 2164260864, 274878038016, 2147614720, 16908288]}, {4: 35461414256640, 3: [35461397479424, 35459266772992, 35186536349696, 277042167808], 2: [35459249995776, 35186519572480, 277025390592, 35184388866048, 274894684160, 2164260864]}, {4: 8454660, 3: [8454656, 8454148, 8389124, 66052], 2: [8454144, 8389120, 66048, 8388612, 65540, 516]}, {4: 1082196480, 3: [1082195968, 1082130944, 1073807872, 8454656], 2: [1082130432, 1073807360, 8454144, 1073742336, 8389120, 66048]}, {4: 138521149440, 3: [138521083904, 138512760832, 137447407616, 1082195968], 2: [138512695296, 137447342080, 1082130432, 137439019008, 1073807360, 8454144]}, {4: 17730707128320, 3: [17730698739712, 17729633386496, 17593268174848, 138521083904], 2: [17729624997888, 17593259786240, 138512695296, 17592194433024, 137447342080, 1082130432]}, {4: 4227330, 3: [4227328, 4227074, 4194562, 33026], 2: [4227072, 4194560, 33024, 4194306, 32770, 258]}, {4: 541098240, 3: [541097984, 541065472, 536903936, 4227328], 2: [541065216, 536903680, 4227072, 536871168, 4194560, 33024]}, {4: 69260574720, 3: [69260541952, 69256380416, 68723703808, 541097984], 2: [69256347648, 68723671040, 541065216, 68719509504, 536903680, 4227072]}, {4: 8865353564160, 3: [8865349369856, 8864816693248, 8796634087424, 69260541952], 2: [8864812498944, 8796629893120, 69256347648, 8796097216512, 68723671040, 541065216]}, {4: 2113665, 3: [2113664, 2113537, 2097281, 16513], 2: [2113536, 2097280, 16512, 2097153, 16385, 129]}, {4: 270549120, 3: [270548992, 270532736, 268451968, 2113664], 2: [270532608, 268451840, 2113536, 268435584, 2097280, 16512]}, {4: 34630287360, 3: [34630270976, 34628190208, 34361851904, 270548992], 2: [34628173824, 34361835520, 270532608, 34359754752, 268451840, 2113536]}, {4: 4432676782080, 3: [4432674684928, 4432408346624, 4398317043712, 34630270976], 2: [4432406249472, 4398314946560, 34628173824, 4398048608256, 34361835520, 270532608]}, {4: 8521760, 3: [8521728, 8519712, 8390688, 133152], 2: [8519680, 8390656, 133120, 8388640, 131104, 2080]}, {4: 1090785280, 3: [1090781184, 1090523136, 1074008064, 17043456], 2: [1090519040, 1074003968, 17039360, 1073745920, 16781312, 266240]}, {4: 139620515840, 3: [139619991552, 139586961408, 137473032192, 2181562368], 2: [139586437120, 137472507904, 2181038080, 137439477760, 2148007936, 34078720]}, {4: 17871426027520, 3: [17871358918656, 17867131060224, 17596548120576, 279239983104], 2: [17867063951360, 17596481011712, 279172874240, 17592253153280, 274945015808, 4362076160]}, {4: 4260880, 3: [4260864, 4259856, 4195344, 66576], 2: [4259840, 4195328, 66560, 4194320, 65552, 1040]}, {4: 545392640, 3: [545390592, 545261568, 537004032, 8521728], 2: [545259520, 537001984, 8519680, 536872960, 8390656, 133120]}, {4: 69810257920, 3: [69809995776, 69793480704, 68736516096, 1090781184], 2: [69793218560, 68736253952, 1090519040, 68719738880, 1074003968, 17039360]}, {4: 8935713013760, 3: [8935679459328, 8933565530112, 8798274060288, 139619991552], 2: [8933531975680, 8798240505856, 139586437120, 8796126576640, 137472507904, 2181038080]}, {4: 2130440, 3: [2130432, 2129928, 2097672, 33288], 2: [2129920, 2097664, 33280, 2097160, 32776, 520]}, {4: 272696320, 3: [272695296, 272630784, 268502016, 4260864], 2: [272629760, 268500992, 4259840, 268436480, 4195328, 66560]}, {4: 34905128960, 3: [34904997888, 34896740352, 34368258048, 545390592], 2: [34896609280, 34368126976, 545259520, 34359869440, 537001984, 8519680]}, {4: 4467856506880, 3: [4467839729664, 4466782765056, 4399137030144, 69809995776], 2: [4466765987840, 4399120252928, 69793218560, 4398063288320, 68736253952, 1090519040]}, {4: 16843009, 3: [16843008, 16842753, 16777473, 65793], 2: [16842752, 16777472, 65792, 16777217, 65537, 257]}, {4: 2155905152, 3: [2155905024, 2155872384, 2147516544, 8421504], 2: [2155872256, 2147516416, 8421376, 2147483776, 8388736, 32896]}, {4: 275955859456, 3: [275955843072, 275951665152, 274882117632, 1077952512], 2: [275951648768, 274882101248, 1077936128, 274877923328, 1073758208, 4210688]}, {4: 35322350010368, 3: [35322347913216, 35321813139456, 35184911056896, 137977921536], 2: [35321811042304, 35184908959744, 137975824384, 35184374185984, 137441050624, 538968064]}, {4: 33686018, 3: [33686016, 33685506, 33554946, 131586], 2: [33685504, 33554944, 131584, 33554434, 131074, 514]}, {4: 4311810304, 3: [4311810048, 4311744768, 4295033088, 16843008], 2: [4311744512, 4295032832, 16842752, 4294967552, 16777472, 65792]}, {4: 551911718912, 3: [551911686144, 551903330304, 549764235264, 2155905024], 2: [551903297536, 549764202496, 2155872256, 549755846656, 2147516416, 8421376]}, {4: 70644700020736, 3: [70644695826432, 70643626278912, 70369822113792, 275955843072], 2: [70643622084608, 70369817919488, 275951648768, 70368748371968, 274882101248, 1077936128]}, {4: 67372036, 3: [67372032, 67371012, 67109892, 263172], 2: [67371008, 67109888, 263168, 67108868, 262148, 1028]}, {4: 8623620608, 3: [8623620096, 8623489536, 8590066176, 33686016], 2: [8623489024, 8590065664, 33685504, 8589935104, 33554944, 131584]}, {4: 1103823437824, 3: [1103823372288, 1103806660608, 1099528470528, 4311810048], 2: [1103806595072, 1099528404992, 4311744512, 1099511693312, 4295032832, 16842752]}, {4: 141289400041472, 3: [141289391652864, 141287252557824, 140739644227584, 551911686144], 2: [141287244169216, 140739635838976, 551903297536, 140737496743936, 549764202496, 2155872256]}]
    def num23(state, mark):
        n12, n13, n22, n23 = 0, 0, 0, 0
        s1, s2 = state[mark], state[3-mark]
        for m in MASKS:
            w1 = s1 & m[4]
            w2 = s2 & m[4]
            if not w2 and w1:
                for m3 in m[3]:
                    if w1 == m3:
                        n13 += 1
                for m2 in m[2]:
                    if w1 == m2:
                        n12 += 1
            if not w1 and w2:
                for m3 in m[3]:
                    if w2 == m3:
                        n23 += 1
                for m2 in m[2]:
                    if w2 == m2:
                        n22 += 1
        return  n12, n13, n22, n23

    def doublewin(state, mark):
        win = 0
        for col in valid_moves_list(state):
            next_state = make_move(state,col, mark)
            if connected_four(next_state[mark]):
                next_state_opp = make_move(state, col, 3-mark)
                if connected_four(make_move(next_state_opp, col, mark)[mark]):
                    return True # win with this column first or second move, unstoppable
                win += 1
                if win == 2:
                    return True # more than one winning move
        return False
    
    def count_even_odd(state, mark):
        # helper for heuristic
        # calculate number of marks in even and odd rows
        odd, even = 0, 0
        for row in range(R):
            rd2 = row%2     
            for col in range(C):
                if (state[mark] >> (C*col+row)) & 1:
                    if rd2 == 0:
                        odd += 1
                    else:
                        even += 1
        return even, odd 

    def heuristic(state, mark):
        # the heuristic uses numbers of 2-marks and 3 -marks windows
        # numbers of marks in the even and odd rows
        # and if player or opponent has "doublewin"

        num_twos, num_threes, num_twos_opp, num_threes_opp = num23(state, mark)
        score = 900000*num_threes - 900000*num_threes_opp + 30000*num_twos - 30000*num_twos_opp

        num_evens, num_odds = count_even_odd(state, mark)
        if first: 
            even_odd_rate = num_odds - num_evens
        else:
            even_odd_rate = num_evens - num_odds
        score += 100 * even_odd_rate

        if num_threes > 1:
            if doublewin(state, mark) : score = 1e8
        if num_threes_opp > 1:
            if doublewin(state, 3-mark) : score = -1e8
        return score 

    def score_move(state, col, mark, nsteps):
        next_state = make_move(state, col, mark)
        return minimax(next_state, nsteps-1, -np.Inf, np.Inf, False, mark)
       
    # Minimax implementation
    def minimax(state, depth, a, b, maximizingPlayer, mark):
        if time.time() > FINISH: # timeout
            return 0
        if connected_four(state[mark]): # player win
            return np.inf
        if connected_four(state[3-mark]): # opponent win
            return -np.inf
        if state[1] | state[2] == 279258638311359: # fullboard mask, tie
            return 0

        if depth == 0: # leaf node
            key = (state[1], state[2]) # the key for store state in the dict
            if key in STATES: # heuristic for the state is in tne dict
                return STATES[key] # get it from the dict
            # new state, calculate heuristic
            h = heuristic(state, mark)
            STATES[key] = h # and store it in the dict
            return h
 
        # MiniMax with alpha-beta pruning
    
        valid_moves = valid_moves_list(state)
       
        if maximizingPlayer:
            value = -np.Inf
            for col in valid_moves:
                child = make_move(state, col, mark)
                value = max(value, minimax(child, depth-1, a, b, False, mark))
                if value >= b:
                    break
                a = max(a, value)
            return value
        else:
            value = np.Inf
            for col in valid_moves:
                child = make_move(state, col, 3-mark)
                value = min(value, minimax(child, depth-1, a, b, True, mark))
                if value <= a:
                    break
                b = min(b, value)
            return value 
    
    # The agent begins to work
    
    START = time.time()
    FINISH = START + DELAY
    
    # the dict to store heuristics, a la transposition table
    STATES = {}  
    
    # the depth to start with
    N_STEPS = 5
    
    # Convert the board to a 2D grid and then the grig  to state
    grid = np.asarray(obs.board).reshape(R, C)
    state = grid_to_state(grid)
    
    # the best move for the first move is "3", let's play it
    step = step_number(grid)
    if step < 2:
        return 3
    
    # remember if the agent is the first player, for heuristic
    first = is_first_player(step)

    valid_moves = valid_moves_list(state)
    # check if we have winning move
    for col in valid_moves:
        next_state = make_move(state, col, obs.mark)
        if connected_four(next_state[obs.mark]):
            # yes, we have, let's play it
            return col
        
    # Use the heuristic to assign a score to each possible board in the next step
    # the agent is rather fast, so it starts with N_STEPS steps to look ahead
    scores = dict(zip(valid_moves, [score_move(state, col, obs.mark, N_STEPS) for col in valid_moves]))

    # Repeat with one step more depth
    while  N_STEPS < 42 - step:
        N_STEPS += 1

        # no need to calculate scores for the moves we already know are the way to loss
        for vm in valid_moves:
            if scores[vm] == -np.inf:
                valid_moves.remove(vm)
        if len(valid_moves) == 0: # we loss anyway, surrender :(
            break
            
        scores_2 = {}
        for col in valid_moves:
            score = score_move(state, col, obs.mark, N_STEPS)
            if score == np.inf:
                # we have the move to win with, let's play it
                return col
            scores_2[col] = score

        if time.time() > FINISH or np.amax(list(scores_2.values())) <= -np.inf:
            # timeout, the search with the next depth isn't complete
            # leave the previous scores as is
            break
        # update the scores withe the result of the next depth search
        scores = scores_2.copy()
    
    # Get a list of columns (moves) that maximize the heuristic
    max_cols = [key for key in scores.keys() if scores[key] == max(scores.values())]

    # Select the best as nearest to the center of the board from the maximizing columns
    if len(max_cols) >0:
        if 3 in max_cols:
            return 3
        if 2 in max_cols and 4 in max_cols:
            return random.choice([2, 4])
        if 2 in max_cols:
            return 2
        if 4 in max_cols:
            return 4
        
        if 1 in max_cols and 5 in max_cols:
            return random.choice([1, 5])
        if 1 in max_cols:
            return 1
        if 5 in max_cols:
            return 5
        
        if 0 in max_cols and 6 in max_cols:
            return random.choice([0, 6])
        if 0 in max_cols:
            return 0
        if 6 in max_cols:
            return 6
        
        col = random.choice(max_cols)
        return col
    col =  random.choice(max_cols)
    return col 

Writing agent_The_Fast_mini_Max_4.py


In [16]:
%run agent_The_Fast_mini_Max_4.py

### [debugger](https://www.kaggle.com/vyacheslavbolotin/debugger-c4)

In [17]:
def debugger_c4(list_of_agents_to_trace):
    map4q = '''
         01,02,03,04; 31,32,33,34;                 32,33,34,35; 33,34,35,36; 
           40,41,42,43; 41,42,43,44;             42,43,44,45; 43,44,45,46; 
             50,51,52,53; 51,52,53,54;         52,53,54,55; 30,31,32,33;
               04,14,24,34; 14,24,34,44;     24,34,44,54; 05,15,25,35; 
                 06,16,26,36; 03,12,21,30; 15,25,35,45; 25,35,45,55; 
                   16,26,36,46; 26,36,46,56;          23,34,45,56;
                     04,13,22,31; 13,22,31,40;      03,14,25,36; 
                       05,14,23,32; 14,23,32,41;  20,30,40,50; 
                         06,15,24,33; 15,24,33,42;
                           24,33,42,51; 23,32,41,50;
                             03,13,23,33; 13,23,33,43; 
                               16,25,34,43; 25,34,43,52; 
                                 11,22,33,44; 11,21,31,41;
                     01,12,23,34;  02,13,24,35; 12,23,34,45; 
                   00,11,22,33;      26,35,44,53; 23,33,43,53;              
                 02,12,22,32;          21,32,43,54; 22,33,44,55; 
               00,10,20,30; 22,23,24,25; 21,31,41,51; 13,24,35,46;
             01,11,21,31; 10,20,30,40;     12,22,32,42; 22,32,42,52;
           00,01,02,03; 10,21,32,43;         02,03,04,05; 03,04,05,06;
         10,11,12,13; 11,12,13,14;             12,13,14,15; 13,14,15,16;
       20,21,22,23; 21,22,23,24;                 23,24,25,26; 20,31,42,53; 53,54,55,56'''
    
    # -----------------------------------------------------------------------------------------------
    start_games_at_away_first_agent = False # True # => The first agent from the list will start his game either “from home” or “away”,
    only_observation_first_agent    = False # True # => Everyone will play two games with the first: one “at home”, the other “away”, or everyone will play with everyone
    # -----------------------------------------------------------------------------------------------
    class Obs:
        def __init__(self, board, mark):
            self.board = board
            self.mark = mark
    # -----------------------------------------------------------------------------------------------
    class Config:
        def __init__(self, rows, columns, inarow):
            self.rows = rows
            self.columns = columns
            self.inarow = inarow
    # -----------------------------------------------------------------------------------------------
    import time
    import numpy as np
    # -----------------------------------------------------------------------------------------------
    Rows, Cols = 6, 7
    config = Config(Rows, Cols, 4)
    AT2 = (np.array([a for a in range(42)])).reshape(Rows, Cols)
    # -----------------------------------------------------------------------------------------------
    def yx(s): return AT2[int(s[0])][int(s[1])]
    Ss = [x.split(",") for x in [x for x in map4q.replace(" ", "").replace("\n", "").split(";")]]
    iMap4q = [list(map(yx, s)) for s in Ss]
    # -----------------------------------------------------------------------------------------------
    def gyx(s): return [int(s[0]), int(s[1])]
    gSs = [x.split(",") for x in [x for x in map4q.replace(" ", "").replace("\n", "").split(";")]]
    gMap4q = [list(map(gyx, s)) for s in gSs]
    # -----------------------------------------------------------------------------------------------
    def inject(col, mark, board):
        i_krai = config.columns * (config.rows - 1) + col
        for i in range(i_krai, -1, -7):
            if board[i] == 0:
                new_board = board.copy()
                new_board[i] = mark
                return new_board
        return board
    # -----------------------------------------------------------------------------------------------
    def win_4o_in(board):
        grid = np.asarray(board).reshape(6,7)
        for i in gMap4q:
            v = grid[i[0][0],i[0][1]] * grid[i[1][0],i[1][1]] * grid[i[2][0],i[2][1]] * grid[i[3][0],i[3][1]]
            if v == 1 or v == 16: return True
        return False
    # -----------------------------------------------------------------------------------------------
    def tsum(reagent):
        home = sum(reagent["games"][0]) if len(reagent["games"][0])>0 else 0
        away = sum(reagent["games"][1]) if len(reagent["games"][1])>0 else 0
        return home + away
    # -----------------------------------------------------------------------------------------------
    def ttime(reagent, gt):
        time = reagent["time"][0]  if len(reagent["time"])>0 else 0
        return [round(time+gt, 2)]
    # -----------------------------------------------------------------------------------------------
    def tmoves(reagent, ms):
        moves = reagent["moves"][0] if len(reagent["moves"])>0 else 0
        return [moves+ms]
    # -----------------------------------------------------------------------------------------------
    def tper1move(reagent):
        time = reagent["time"][0]   if len(reagent["time"]) > 0 else 0
        moves = reagent["moves"][0] if len(reagent["moves"])> 0 else 0
        return [round(time / moves, 1)]
    # -----------------------------------------------------------------------------------------------
    def print_to_clip(board, ims, i, attack, defense, stime1, stime2, s1, s2):
        print("1.{0} [{1}]   2.{2} [{3}]".format(name(attack), s1, name(defense), s2),
              "  1.Time =", round(stime1, 2), '  2.Time =', round(stime2, 2), '  q.moves:', i,
              "  1.speed =", round(stime1 / i, 1), "  2.speed =", round(stime2 / i, 1),
              "  \n\nGame:", str(ims).replace(", ", ","),
              "  \n\nBoard:", str(board).replace(", ", ","),"\n")
        bn2d = np.array(board).reshape(Rows, Cols)
        for r in range(len(bn2d)): print(bn2d[r])
        print('{0}{1}'.format('\n',"="*101))
    # -----------------------------------------------------------------------------------------------
    def rec(total_points, _A, _D, ia, id, stime1, stime2):
        reatta = total_points[name(Attack)]
        redefe = total_points[name(Defense)]
        reatta["games"][0].extend([_A])
        redefe["games"][1].extend([_D])
        reatta["T"] = [tsum(reatta)]
        redefe["T"] = [tsum(redefe)]
        reatta["time"] = ttime(reatta, stime1)
        redefe["time"] = ttime(redefe, stime2)
        reatta["moves"] = tmoves(reatta, ia)
        redefe["moves"] = tmoves(redefe, id)
        reatta["speed"] = tper1move(reatta)
        redefe["speed"] = tper1move(redefe)
    # -----------------------------------------------------------------------------------------------
    def agent_work(Agent, board, mark, config):
        obs = Obs(board, mark)
        start = time.time()
        im = Agent(obs, config)
        t = time.time() - start
        board_next = inject(im, obs.mark, board)
        return im, t, board_next
    # -----------------------------------------------------------------------------------------------
    def write_Connect4(ims):
        #     with open("Connect4.txt", "w") as file:
        #         file.write(str(ims))
        pass 
    # -----------------------------------------------------------------------------------------------
    def write_uraConnect(ura_ims, Attack, Defense):
        #     try:
        #         with open("uraConnect.txt", "w") as file:
        #             stf = str(ura_ims) + '\n' + name(Attack) + ' - ' + name(Defense)
        #             file.write(stf)
        #     finally: return
        pass
    # -----------------------------------------------------------------------------------------------
    def name(agent): return agent.__name__.replace("my_","")
    # -----------------------------------------------------------------------------------------------

    Agents = list_of_agents_to_trace # [ag1,ag2,..,agn]

    observatory = Agents[0] if only_observation_first_agent and len(Agents) > 1 else None

    total_points = { name(agent):{"T":[],"games":[[],[]],"time":[],"moves":[],"speed":[]} for agent in Agents }

    ig, board__0 = 1, [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

    for home in Agents:
        for away in Agents:
            if home!=away and (observatory==None or observatory!=None and (observatory==home or observatory==away)):
                if ig==1:
                    print("\n")
                    print(ig-1, ":")
                    print('----------------')
                    for itp in total_points.items(): print(itp)
                    ig += 1
                stime1, stime2, imts1, imts2, ims, imt, = 0, 0, [], [], [], [[], []]
                board__1 = board__0
                # <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
                if start_games_at_away_first_agent:
                    Attack, Defense = away, home
                else:
                    Attack, Defense = home, away
                # >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
                total_points_Attack, total_points_Defense = [], []
                print("--------------------------")
                print(Attack.__name__, "  vs  ", Defense.__name__)
                print("-----------------------------------")
                for i in range(1,22):
                    im1, t1, board__2  = agent_work(Attack, board__1, 1, config)
                    stime1 +=t1
                    imt[0] = str(round(t1,0))
                    ims.append(im1)
                    imts1.append(imt[0])
                    write_uraConnect (ims, Attack, Defense)
                    if win_4o_in(board__2):
                        rec(total_points, 3, 0, i, i-1, stime1, stime2)
                        print('\nwin in n win in n win in n win in n win in n win in n win in n win in n')
                        print('   1 1  1   1    1     1     ', Attack.__name__, "     1     1    1   1  1 1")
                        print('win in n win in n win in n win in n win in n win in n win in n win in n\n')
                        print_to_clip (board__2, ims, i, Attack, Defense, stime1, stime2, 3,0)
                        write_Connect4 (ims)
                        write_uraConnect (ims, Attack, Defense)
                        time.sleep(1)
                        break
                    im2, t2, board__1 = agent_work(Defense, board__2, 2, config)
                    stime2 +=t2
                    imt[1] = str(round(t2, 0))
                    ims.append(im2)
                    imts2.append(imt[1])
                    print('({0}) . {1} > {2}  {3} < {4} . ({5}) --- {6} : {7} '
                          .format(round(t1,1), name(Attack), im1, im2, name(Defense), round(t2,1), i, str(ims).replace(", ",",")))
                    write_uraConnect (ims, Attack, Defense)
                    if win_4o_in(board__1):
                        rec(total_points, 0, 3, i, i, stime1, stime2)
                        print('\nwin in n win in n win in n win in n win in n win in n win in n win in n')
                        print('   2 2  2   2    2     2     ', Defense.__name__,"     2     2    2   2  2 2")
                        print('win in n win in n win in n win in n win in n win in n win in n win in n\n')
                        print_to_clip(board__1, ims, i, Attack, Defense, stime1, stime2, 0,3)
                        write_Connect4 (ims)
                        write_uraConnect(ims, Attack, Defense)
                        time.sleep(1)
                        break
                if not win_4o_in(board__1) and not win_4o_in(board__2):
                    rec(total_points, 1, 2, i, i, stime1, stime2)
                    print('\nDRAW RAW AW W DRAW RAW AW W DRAW RAW AW W DRAW RAW AW W DRAW RAW AW W DRAW RAW AW W')
                    print('  0 0  0   0    0     ', Attack.__name__, " - ", Defense.__name__, "     0    0   0  0 0")
                    print('DRAW RAW AW W DRAW RAW AW W DRAW RAW AW W DRAW RAW AW W DRAW RAW AW W DRAW RAW AW W\n')
                    print_to_clip(board__1, ims, i, Attack, Defense, stime1, stime2, 1,2)
                    write_Connect4 (ims)
                    write_uraConnect(ims, Attack, Defense)
                    time.sleep(3)
                print("\n")
                print(ig,":\n") # , ": ", total_points,
                for itp in total_points.items():
                    print(itp)
                print("\n")
                ig +=1

In [18]:
debugger_c4([
    agent__treebased__dott__,
    agent_MCTS_MatanTsipory_,
    MCTS_wAdaptive_Playouts_,
    MontyCarlo_JamesMcGuigan,
    MontyCarlo_BtS_JMcGuigan,
    agent_TheFast_mini_Max_2,
    agent_TheFast_mini_Max_4,
])



0 :
----------------
('agent__treebased__dott__', {'T': [], 'games': [[], []], 'time': [], 'moves': [], 'speed': []})
('agent_MCTS_MatanTsipory_', {'T': [], 'games': [[], []], 'time': [], 'moves': [], 'speed': []})
('MCTS_wAdaptive_Playouts_', {'T': [], 'games': [[], []], 'time': [], 'moves': [], 'speed': []})
('MontyCarlo_JamesMcGuigan', {'T': [], 'games': [[], []], 'time': [], 'moves': [], 'speed': []})
('MontyCarlo_BtS_JMcGuigan', {'T': [], 'games': [[], []], 'time': [], 'moves': [], 'speed': []})
('agent_TheFast_mini_Max_2', {'T': [], 'games': [[], []], 'time': [], 'moves': [], 'speed': []})
('agent_TheFast_mini_Max_4', {'T': [], 'games': [[], []], 'time': [], 'moves': [], 'speed': []})
--------------------------
agent__treebased__dott__   vs   agent_MCTS_MatanTsipory_
-----------------------------------
(0.1) . agent__treebased__dott__ > 3  2 < agent_MCTS_MatanTsipory_ . (1.0) --- 1 : [3,2] 
(0.1) . agent__treebased__dott__ > 3  3 < agent_MCTS_MatanTsipory_ . (1.0) --- 2 : [3,2,

## bot battle

In [19]:
# round_robin(agents=[
#     agent__treebased__dott__,
#     agent_MCTS_MatanTsipory_,
#     MCTS_wAdaptive_Playouts_,
# ], n_games=2)

In [20]:
# round_robin(agents=[
#     agent__treebased__dott__,
#     MontyCarlo_JamesMcGuigan,
#     MontyCarlo_BtS_JMcGuigan,
# ], n_games=2)

In [21]:
# round_robin(agents=[
#     agent__treebased__dott__,
#     agent_TheFast_mini_Max_2,
#     agent_TheFast_mini_Max_4,
# ], n_games=2)